In [ ]:
# ====================================================================================================
#   IMPORTS
# ====================================================================================================
import discord
import aiohttp
import datetime
import time
import platform
import asyncio
import random
import io
import os
import re
import logging
import urllib.parse
import aiosqlite # Para tokens do Spotify
import json
import traceback
import math # Adicionado para XP
import pymongo # Para o ReturnDocument
import motor.motor_asyncio # Driver Assíncrono (para Bot)

from discord.ext import commands, tasks
from discord import app_commands, ui
from discord import Embed
from datetime import datetime, timedelta, timezone as dt_timezone
from discord.ui import Button, View, Select, UserSelect, RoleSelect
from typing import Optional, Dict, Any
from io import BytesIO
import google.generativeai as genai
from dotenv import load_dotenv
from aiohttp import web # Para o servidor web do Spotify

try:
    from PIL import Image, ImageDraw, ImageFont
except ImportError:
    raise RuntimeError("Pillow não está instalado. Use: pip install Pillow")

# --- IMPORTS MONGODB ---
try:
    # Importa as funções e variáveis do nosso gerenciador de conexão
    from mongo_utils import profiles_async, get_profile_bot, async_client, db_async
except ImportError:
    print("ERRO CRÍTICO: mongo_utils.py não encontrado.")
    # Idealmente, o bot não deve iniciar sem isso
    # exit(1)
except Exception as e:
    print(f"ERRO CRÍTICO ao importar mongo_utils (Bot): {e}")
    # exit(1)

# ====================================================================================================
#   CARREGAR VARIÁVEIS DE AMBIENTE E CONFIGURAÇÃO
# ====================================================================================================
load_dotenv()

# --- Discord Bot ---
DISCORD_TOKEN = os.getenv("DISCORD_TOKEN")
APP_ID = os.getenv("APP_ID")
OWNER_ID_STR = os.getenv("OWNER_ID")

# --- Ticket/Perfil/Moderação ---
GUILD_ID_PRINCIPAL = os.getenv('GUILD_ID_PRINCIPAL')
ID_CARGO_ATENDENTE = os.getenv('ID_CARGO_ATENDENTE')
MUTED_ROLE_NAME = os.getenv("MUTED_ROLE_NAME", "Muted")
MOD_LOG_CHANNEL_ID_STR = os.getenv("MOD_LOG_CHANNEL_ID")

# --- Google AI (Gemini) ---
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# --- Spotify API ---
SPOTIFY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
REDIRECT_URI = os.getenv("SPOTIFY_CALLBACK_URL", "http://localhost:8080/callback") # Callback do Spotify

# --- Validações Críticas ---
if not all([DISCORD_TOKEN, APP_ID, GOOGLE_API_KEY, OWNER_ID_STR, SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET, GUILD_ID_PRINCIPAL, ID_CARGO_ATENDENTE]):
    raise ValueError("Variáveis .env essenciais não definidas.")

try:
    OWNER_ID = int(OWNER_ID_STR)
    # GUILD_ID_OBJ é usado para registrar comandos. Se for para todos os servidores, comente a linha
    GUILD_ID_OBJ = discord.Object(id=GUILD_ID_PRINCIPAL)
    MOD_LOG_CHANNEL_ID = int(MOD_LOG_CHANNEL_ID_STR) if MOD_LOG_CHANNEL_ID_STR else None
except ValueError as e:
    raise ValueError(f"ID inválido no .env: {e}")

# --- Configuração das APIs ---
try:
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-1.5-flash') # Modelo atualizado
except Exception as e:
    raise RuntimeError(f"Erro ao configurar API Google: {e}")

# --- Configurações Adicionais ---
OAUTH_SCOPES = ["user-read-playback-state", "user-modify-playback-state", "user-read-currently-playing", "playlist-read-private", "user-read-private", "user-top-read"]
DATABASE_PATH = "spotify_tokens.db" # Renomeado para clareza, profiles.json não é mais usado
WEB_SERVER_HOST = "0.0.0.0"
WEB_SERVER_PORT = int(os.getenv("PORT", 8080))

# Caminhos relativos (assumindo que estão na mesma pasta do main.py)
IMAGE_FOLDER = "profile_fotos"
SELFIE_IMAGE_FOLDER = "selfie_images"

ROLE_BACKGROUNDS = {int(k): v for k, v in {
    "1309742404508581988": "fundo-classic.png",
    "1309742612722225173": "fundo_divine.png",
    "1309742657555140669": "fundo-supreme.png",
    "1309287632273936462": "fundo-booster.png",
}.items()}
STATUS_VALIDOS = ["Disponível", "Ausente", "Ocupado"]
LOCALIZACOES_VALIDAS = ["brasil", "portugal", "estados unidos", "japao", "canada", "albania"]
ACTION_NAMES = {"disconnect": "Desconectar da call", "ban": "Banir", "kick": "Expulsar (kick)", "mute_role": "Silenciar (por cargo)", "remove_roles": "Remover todos os cargos"}
SELFIE_IMAGE_SETTINGS = {
    3: {"template": "base_3_{variant}.png", "positions": { 1: [(200, 200), (440, 215), (670, 145)], 2: [(400, 251), (800, 685), (1510, 430)], 3: [(160, 200), (340, 70), (560, 130)], 4: [(60, 50), (255, 60), (450, 90)]}, "avatar_size": (150, 150)},
    4: {"template": "base_4_{variant}.png", "positions": { 1: [(80, 120), (220, 300), (350, 150), (180, 400)], 2: [(120, 80), (250, 200), (400, 300), (100, 350)], 3: [(200, 150), (100, 300), (300, 200), (400, 100)], 4: [(150, 250), (280, 150), (50, 300), (350, 200)]}, "avatar_size": (110, 110)},
    5: {"template": "base_5_{variant}.png", "positions": { 1: [(50, 100), (200, 200), (350, 150), (150, 300), (400, 250)], 2: [(100, 80), (250, 180), (400, 120), (180, 280), (300, 350)], 3: [(120, 150), (280, 200), (200, 300), (350, 100), (80, 250)], 4: [(150, 100), (300, 200), (50, 300), (250, 150), (400, 350)]}, "avatar_size": (100, 100)}
}
multimidia = ["https://media0.giphy.com/media/11rWoZNpAKw8w/giphy.gif", "https://media0.giphy.com/media/BXrwTdoho6hkQ/giphy.gif", "https://media0.giphy.com/media/Y8wCpaKI9PUBO/giphy.gif"]
frases_de_amor = ["Você é a razão do meu sorriso!", "Cada momento ao seu lado é mágico!", "Meu coração bate mais forte com você!", "Amar você é a melhor parte do dia!", "Você transforma meus dias em poesia.", "Você é meu sonho realizado.", "Com você, tudo é mais doce.", "Meu amor por você é infinito."]

# --- Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s:%(levelname)s:%(name)s: %(message)s')
logger = logging.getLogger("discord-bot")

# --- Exceptions ---
class SpotifyAuthError(Exception): pass
class SpotifyAPIError(Exception): pass

# ====================================================================================================
#   HELPERS (Spotify Tokens, Imagens)
# ====================================================================================================

# --- Banco de Dados (aiosqlite) - APENAS PARA TOKENS DO SPOTIFY ---
async def init_spotify_db():
    async with aiosqlite.connect(DATABASE_PATH) as db:
        await db.execute("CREATE TABLE IF NOT EXISTS tokens (discord_id TEXT PRIMARY KEY, access_token TEXT NOT NULL, refresh_token TEXT NOT NULL, expires_at REAL NOT NULL, scope TEXT, token_type TEXT);")
        await db.commit()

async def save_tokens_for_user(discord_id: str, token_data: Dict[str, Any]):
    expires_at = time.time() + int(token_data["expires_in"])
    async with aiosqlite.connect(DATABASE_PATH) as db:
        await db.execute("INSERT INTO tokens VALUES (?, ?, ?, ?, ?, ?) ON CONFLICT(discord_id) DO UPDATE SET access_token=excluded.access_token, refresh_token=excluded.refresh_token, expires_at=excluded.expires_at, scope=excluded.scope, token_type=excluded.token_type;",
                         (discord_id, token_data["access_token"], token_data["refresh_token"], expires_at, token_data.get("scope", ""), token_data.get("token_type", "Bearer")))
        await db.commit()
    logger.info("Tokens Spotify salvos para %s", discord_id)

async def get_tokens_for_user(discord_id: str) -> Optional[Dict[str, Any]]:
    async with aiosqlite.connect(DATABASE_PATH) as db:
        db.row_factory = aiosqlite.Row
        async with db.execute("SELECT * FROM tokens WHERE discord_id = ?", (discord_id,)) as cursor: row = await cursor.fetchone()
    return dict(row) if row else None

async def refresh_access_token_if_needed(session: aiohttp.ClientSession, discord_id: str) -> str:
    tokens = await get_tokens_for_user(discord_id)
    if not tokens: raise SpotifyAuthError("Usuário não autenticado com Spotify.")
    if tokens["expires_at"] - time.time() > 60: return tokens["access_token"]

    logger.info("Token Spotify para %s expirado, renovando...", discord_id)
    data = {"grant_type": "refresh_token", "refresh_token": tokens["refresh_token"]}
    auth = aiohttp.BasicAuth(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)
    try:
        async with session.post("https://accounts.spotify.com/api/token", data=data, auth=auth) as resp:
            resp.raise_for_status()
            payload = await resp.json()
            new_tokens = {
                "access_token": payload["access_token"],
                "refresh_token": payload.get("refresh_token", tokens["refresh_token"]), # Mantém o refresh token antigo se um novo não for enviado
                "expires_in": payload.get("expires_in", 3600),
                "scope": payload.get("scope", tokens.get("scope")),
                "token_type": payload.get("token_type", tokens.get("token_type"))
            }
            await save_tokens_for_user(discord_id, new_tokens)
            return new_tokens["access_token"]
    except aiohttp.ClientResponseError as e:
        logger.error("Falha ao renovar token Spotify %s: %s - %s", discord_id, e.status, e.message)
        if e.status == 400: # 400 Bad Request (often 'invalid_grant')
            raise SpotifyAuthError("Autorização expirada ou revogada. Faça login novamente.")
        raise SpotifyAuthError(f"Falha ao renovar token Spotify (Erro {e.status}).") from e
    except aiohttp.ClientError as e:
        logger.exception("Erro de conexão ao renovar token Spotify.")
        raise SpotifyAPIError("Falha ao conectar na API Spotify.") from e

async def spotify_request(session: aiohttp.ClientSession, discord_id: str, method: str, endpoint: str, **kwargs) -> Any:
    if not session or session.closed:
        logger.error("spotify_request foi chamado com sessão inválida ou fechada.")
        global client
        if not hasattr(client, 'http_session') or not client.http_session or client.http_session.closed:
             logger.critical("A sessão HTTP principal do Bot também está fechada! Não é possível fazer requisições.")
             raise SpotifyAPIError("Internal bot error: HTTP session not available.")
        logger.warning("Usando a sessão HTTP do client como fallback.")
        session = client.http_session

    try:
        access_token = await refresh_access_token_if_needed(session, discord_id)
    except (SpotifyAuthError, SpotifyAPIError) as e:
        raise e # Propaga o erro específico
    except Exception as e:
         logger.error(f"Erro inesperado ao tentar renovar token: {e}", exc_info=True)
         raise SpotifyAPIError("Erro interno ao renovar token Spotify.")

    headers = kwargs.pop("headers", {"Authorization": f"Bearer {access_token}", "Content-Type": "application/json"})
    url = f"https://api.spotify.com/v1{endpoint}"

    try:
        async with session.request(method, url, headers=headers, **kwargs) as resp:
            if resp.status == 204: return None # No Content

            # Tenta ler o corpo da resposta ANTES de checar o status >= 400
            # Apenas lê se soubermos que não é 204
            response_data = None
            try:
                if "application/json" in resp.headers.get("Content-Type", ""):
                    response_data = await resp.json()
                else:
                    response_data = await resp.text()
            except (aiohttp.ContentTypeError, json.JSONDecodeError):
                response_data = await resp.text() # Fallback para texto se o JSON falhar

            if resp.status >= 400:
                logger.error("Erro API Spotify [%s %s]: %s - %s", method, url, resp.status, response_data)

                if resp.status == 401:
                    raise SpotifyAuthError("Token inválido/expirado. Faça login novamente.")
                elif resp.status == 403:
                     # Tenta extrair a razão do erro se a resposta foi JSON
                     reason = "Unknown Reason"
                     if isinstance(response_data, dict):
                         reason = response_data.get("error", {}).get("reason", "Unknown Reason")

                     if reason == "PREMIUM_REQUIRED":
                           raise SpotifyAPIError("Ação requer Spotify Premium.")
                     elif reason in ("NO_ACTIVE_DEVICE", "PLAYER_COMMAND_FAILED"):
                           raise SpotifyAPIError("Nenhum dispositivo Spotify ativo encontrado ou comando falhou.")
                     else:
                           raise SpotifyAPIError(f"Acesso negado ({reason}). Refaça login para novas permissões.")
                elif resp.status == 404:
                    raise SpotifyAPIError("Recurso não encontrado (Player inativo ou URL errada?).")
                else:
                    raise SpotifyAPIError(f"Erro API Spotify ({resp.status}).")

            return response_data # Retorna o JSON ou texto se o status foi OK

    except aiohttp.ClientConnectorError as e:
        logger.error(f"Erro de conexão com API Spotify: {e}")
        raise SpotifyAPIError("Falha ao conectar na API Spotify (verifique a rede).") from e
    except (SpotifyAuthError, SpotifyAPIError):
         raise # Propaga erros conhecidos
    except Exception as e:
         logger.exception("Erro inesperado durante a requisição Spotify.")
         raise SpotifyAPIError("Erro interno ao fazer requisição Spotify.") from e

# --- Funções de Perfil (MongoDB) ---
# get_profile_bot() é importado de mongo_utils.py

def calculate_level(xp): return int(0.1 * (xp ** 0.5)) if xp > 0 else 0
def xp_for_level(level): return int((level / 0.1) ** 2) if level > 0 else 0

# --- Funções de Imagem (Pillow) ---
async def fetch_image(session: aiohttp.ClientSession, url: str) -> Optional[bytes]:
    """Downloads image data from a URL asynchronously."""
    if not url: return None
    try:
        async with session.get(url, timeout=10) as response:
            if response.status == 200:
                return await response.read()
            else:
                logger.warning(f"Failed to download image from {url}, status: {response.status}")
                return None
    except aiohttp.ClientError as e:
        logger.error(f"Network error downloading image {url}: {e}")
        return None
    except asyncio.TimeoutError:
         logger.warning(f"Timeout downloading image from {url}")
         return None
    except Exception as e:
        logger.error(f"Unexpected error downloading image {url}: {e}")
        return None

def text_wrap(text: str, font: ImageFont.FreeTypeFont, max_width: int) -> str:
    """Wraps text to fit within a maximum width."""
    if not text: return ""
    lines = []
    words = text.split()
    if not words: return ""

    current_line = words[0]
    for word in words[1:]:
        try:
             # Usa textbbox que é mais preciso que getsize
             bbox = font.getbbox(current_line + " " + word)
             line_width = bbox[2] - bbox[0] # width = right_x - left_x
        except Exception:
             # Fallback para getlength se getbbox falhar (versões antigas/fontes)
             line_width = font.getlength(current_line + " " + word)

        if line_width <= max_width:
            current_line += " " + word
        else:
            lines.append(current_line)
            current_line = word
    lines.append(current_line)
    return "\n".join(lines)

# Compatibilidade de resampling entre versões do Pillow
try:
    RESAMPLE = Image.Resampling.LANCZOS
except AttributeError:
    RESAMPLE = Image.LANCZOS # Fallback para Pillow < 9

# --- Geradores de Imagem (Spotify) ---
async def create_spotify_embed_image(session: aiohttp.ClientSession, player_state: Dict[str, Any]) -> BytesIO:
    item = player_state.get("item", {})
    if not item:
        raise ValueError("Nenhuma música a tocar.")

    track_name = item.get("name", "Faixa Desconhecida")
    artists = ", ".join(a.get("name", "") for a in item.get("artists", [])) or "Artista Desconhecido"

    images = item.get("album", {}).get("images", []) or []
    album_art_url = None
    for img in images:
        if img.get("height", 0) > 300: # Pega imagem de boa qualidade
            album_art_url = img.get("url")
            break
    if not album_art_url and images:
        album_art_url = images[0].get("url") # Pega a primeira disponível

    duration_ms = item.get("duration_ms", 1)
    progress_ms = player_state.get("progress_ms", 0)

    def ms_to_time(ms: int) -> str:
        s = int(ms / 1000)
        m, s = divmod(s, 60)
        return f"{m:02d}:{s:02d}"

    WIDTH, HEIGHT = 1116, 458

    if not album_art_url:
        raise ValueError("URL da capa não encontrada.")

    art_bytes = await fetch_image(session, album_art_url)
    if not art_bytes:
        raise ValueError("Falha ao baixar capa.")

    album_art_raw = Image.open(BytesIO(art_bytes)).convert("RGBA")
    art_size = (458, 445)
    album_art = album_art_raw.resize(art_size, RESAMPLE)

    mask = Image.new("L", art_size, 0)
    mask_draw = ImageDraw.Draw(mask)
    mask_draw.rounded_rectangle((0, 0, art_size[0], art_size[1]), radius=30, fill=255)

    background_path = "spotify_background.png"
    if not os.path.exists(background_path):
        raise FileNotFoundError(f"{background_path} não encontrado.")
    background = Image.open(background_path).convert("RGBA")

    canvas = background.copy()
    draw = ImageDraw.Draw(canvas)
    art_pos = (6, 6)
    canvas.paste(album_art, art_pos, mask)

    try:
        font_title = ImageFont.truetype("arialbd.ttf", 48)
        font_artists = ImageFont.truetype("arial.ttf", 36)
        font_small = ImageFont.truetype("arial.ttf", 22)
    except Exception:
        logger.warning("Fontes Arial não encontradas. Usando padrão.")
        font_title = ImageFont.load_default()
        font_artists = ImageFont.load_default()
        font_small = ImageFont.load_default()

    padding = 30
    right_x_start = art_pos[0] + art_size[0] + padding
    max_text_width = WIDTH - right_x_start - padding
    y = padding + 10

    # Título (quebra em até 3 linhas)
    wrapped_title = text_wrap(track_name, font_title, max_text_width)
    wrapped_title = "\n".join(wrapped_title.split('\n')[:3])
    line_spacing = 6
    draw.multiline_text((right_x_start, y), wrapped_title, font=font_title, fill="#FFFFFF", spacing=line_spacing)

    if wrapped_title:
        try:
            bbox = draw.multiline_textbbox((right_x_start, y), wrapped_title, font=font_title, spacing=line_spacing)
            y = bbox[3]
        except Exception:
            lines_count = wrapped_title.count('\n') + 1
            bbox_line = draw.textbbox((0,0), "A", font=font_title)
            line_height = bbox_line[3] - bbox_line[1]
            y += (line_height + line_spacing) * lines_count - line_spacing

    # Artistas (até 2 linhas)
    y += 6
    wrapped_artists = text_wrap(artists, font_artists, max_text_width)
    wrapped_artists = "\n".join(wrapped_artists.split('\n')[:2])
    line_spacing_artists = 4
    draw.multiline_text((right_x_start, y), wrapped_artists, font=font_artists, fill="#B3B3B3", spacing=line_spacing_artists)

    if wrapped_artists:
        try:
            bbox_artists = draw.multiline_textbbox((right_x_start, y), wrapped_artists, font=font_artists, spacing=line_spacing_artists)
            y = bbox_artists[3] + 30
        except Exception:
            lines_count_artists = wrapped_artists.count('\n') + 1
            bbox_line_artists = draw.textbbox((0,0), "A", font=font_artists)
            line_height_artists = bbox_line_artists[3] - bbox_line_artists[1]
            y += ((line_height_artists + line_spacing_artists) * lines_count_artists - line_spacing_artists) + 30
    else:
        y += 30

    # Barra de progresso
    bar_x, bar_y, bar_width, bar_height = right_x_start, y, max_text_width, 8
    progress_ratio = (progress_ms / duration_ms) if duration_ms > 0 else 0
    progress_width = int(bar_width * progress_ratio)
    draw.rounded_rectangle((bar_x, bar_y, bar_x + bar_width, bar_y + bar_height), radius=4, fill="#535353")
    if progress_width > 0:
        draw.rounded_rectangle((bar_x, bar_y, bar_x + progress_width, bar_y + bar_height), radius=4, fill="#1DB954")
    y += bar_height + 8

    # Tempo
    draw.text((right_x_start, y), f"{ms_to_time(progress_ms)} / {ms_to_time(duration_ms)}", font=font_small, fill="#B3B3B3")

    image_buffer = BytesIO()
    canvas.save(image_buffer, format="PNG")
    image_buffer.seek(0)
    return image_buffer

async def create_spotify_share_image(session: aiohttp.ClientSession, player_state: Dict[str, Any], discord_username: str) -> BytesIO:
    item = player_state.get("item", {})
    track_name = item.get("name", "Faixa Desconhecida")
    artists = ", ".join(a.get("name", "") for a in item.get("artists", [])) or "Artista Desconhecido"

    images = item.get("album", {}).get("images", []) or []
    album_art_url = None
    for img in images:
        if img.get("height", 0) > 300:
            album_art_url = img.get("url")
            break
    if not album_art_url and images:
        album_art_url = images[0].get("url")

    device_name = player_state.get("device", {}).get("name", "Spotify")
    duration_ms = item.get("duration_ms", 1)

    def ms_to_time(ms: int) -> str:
        s = int(ms / 1000)
        m, s = divmod(s, 60)
        return f"{m:02d}:{s:02d}"

    SCALE = 0.8
    WIDTH, HEIGHT = int(1116 * SCALE), int(458 * SCALE)

    if not album_art_url:
        raise ValueError("URL da capa não encontrada.")

    art_bytes = await fetch_image(session, album_art_url)
    if not art_bytes:
        raise ValueError("Falha ao baixar capa.")

    album_art_raw = Image.open(BytesIO(art_bytes)).convert("RGBA")

    art_size = (int(458 * SCALE), int(445 * SCALE))
    art_pos = (int(6 * SCALE), int(6 * SCALE))
    mask_radius = int(30 * SCALE)

    album_art = album_art_raw.resize(art_size, RESAMPLE)

    mask = Image.new("L", art_size, 0)
    mask_draw = ImageDraw.Draw(mask)
    mask_draw.rounded_rectangle((0, 0, art_size[0], art_size[1]), radius=mask_radius, fill=255)

    background_path = "spotify_background.png"
    if not os.path.exists(background_path):
        raise FileNotFoundError(f"{background_path} não encontrado.")

    background = Image.open(background_path).convert("RGBA").resize((WIDTH, HEIGHT), RESAMPLE)

    canvas = background.copy()
    draw = ImageDraw.Draw(canvas)
    canvas.paste(album_art, art_pos, mask)

    try:
        font_username = ImageFont.truetype("arial.ttf", int(28 * SCALE))
        font_title = ImageFont.truetype("arialbd.ttf", int(48 * SCALE))
        font_artists = ImageFont.truetype("arial.ttf", int(36 * SCALE))
        font_small = ImageFont.truetype("arial.ttf", int(18 * SCALE))
    except Exception:
        logger.warning("Fontes Arial não encontradas. Usando padrão.")
        font_username = ImageFont.load_default()
        font_title = ImageFont.load_default()
        font_artists = ImageFont.load_default()
        font_small = ImageFont.load_default()

    padding = int(30 * SCALE)
    right_x_start = art_pos[0] + art_size[0] + padding
    max_text_width = WIDTH - right_x_start - padding

    draw.text((right_x_start, padding), f"{discord_username} está ouvindo...", font=font_username, fill="#B3B3B3")
    username_bbox = draw.textbbox((0, 0), f"{discord_username} está ouvindo...", font=font_username)
    y = padding + (username_bbox[3] - username_bbox[1]) + int(8 * SCALE)

    wrapped_title = text_wrap(track_name, font_title, max_text_width)
    wrapped_title = "\n".join(wrapped_title.split('\n')[:3])
    line_spacing = int(6 * SCALE)

    draw.multiline_text((right_x_start, y), wrapped_title, font=font_title, fill="#FFFFFF", spacing=line_spacing)

    if wrapped_title:
        try:
            bbox = draw.multiline_textbbox((right_x_start, y), wrapped_title, font=font_title, spacing=line_spacing)
            y = bbox[3] + int(6 * SCALE)
        except Exception:
            lines_count = wrapped_title.count('\n') + 1
            bbox_line = draw.textbbox((0,0), "A", font=font_title)
            line_height = bbox_line[3] - bbox_line[1]
            y += (line_height + line_spacing) * lines_count - line_spacing + int(6 * SCALE)

    wrapped_artists = text_wrap(artists, font_artists, max_text_width)
    wrapped_artists = "\n".join(wrapped_artists.split('\n')[:2])
    line_spacing_artists = int(4 * SCALE)

    draw.multiline_text((right_x_start, y), wrapped_artists, font=font_artists, fill="#B3B3B3", spacing=line_spacing_artists)

    padding_after_artists = int(10 * SCALE)
    if wrapped_artists:
        try:
            bbox_artists = draw.multiline_textbbox((right_x_start, y), wrapped_artists, font=font_artists, spacing=line_spacing_artists)
            y = bbox_artists[3] + padding_after_artists
        except Exception:
            lines_count_artists = wrapped_artists.count('\n') + 1
            bbox_line_artists = draw.textbbox((0,0), "A", font=font_artists)
            line_height_artists = bbox_line_artists[3] - bbox_line_artists[1]
            y += ((line_height_artists + line_spacing_artists) * lines_count_artists - line_spacing_artists) + padding_after_artists
    else:
         y += padding_after_artists

    draw.text((right_x_start, y), f"{ms_to_time(player_state.get('progress_ms', 0))} / {ms_to_time(duration_ms)}", font=font_username, fill="#FFFFFF")

    footer_pos_x = int(464 * SCALE)
    footer_pos_y = HEIGHT - int(28 * SCALE)
    draw.text((footer_pos_x, footer_pos_y), f"Tocando no {device_name}", font=font_small, fill="#1DB954")

    image_buffer = BytesIO()
    canvas.save(image_buffer, format="PNG")
    image_buffer.seek(0)
    return image_buffer

# --- Gerador de Imagem (Perfil) ---
# **MODIFICADO** para aceitar 'user_profile' como argumento
def generate_profile_image_sync(member: discord.Member, pfp_data: bytes, user_profile: dict) -> BytesIO:
    # user_profile = get_user_profile(getattr(member, "id", 0)) # <-- REMOVIDO!
    user_id_for_log = getattr(member, "id", "UNKNOWN_ID")

    # --- CONSTANTES DE LAYOUT (Ajuste aqui) ---
    CANVAS_WIDTH = 2400
    CANVAS_HEIGHT = 1055

    # --- Fontes ---
    try:
        font_path = "arial.ttf"
        font_path_bold = "arialbd.ttf"
        username_font = ImageFont.truetype(font_path_bold, size=90)
        info_font = ImageFont.truetype(font_path, size=45)
        bio_font = ImageFont.truetype(font_path, size=50)
        level_font = ImageFont.truetype(font_path_bold, size=60)
        xp_font = ImageFont.truetype(font_path, size=35)
        small_info_font = ImageFont.truetype(font_path, size=30)
    except Exception as e:
        logger.warning(f"Fontes Arial não encontradas ({e}). Usando padrão.")
        username_font = info_font = bio_font = level_font = xp_font = small_info_font = ImageFont.load_default()

    # --- Função Auxiliar (load_img) ---
    def load_img(name, resize_to=None, is_essential=False):
        path = os.path.join(IMAGE_FOLDER, name)
        if not os.path.exists(path):
            if is_essential:
                 logger.error(f"Erro CRÍTICO ao carregar '{name}': Arquivo não encontrado em {path}")
                 raise FileNotFoundError(f"{path} não encontrado.")
            else:
                 logger.warning(f"Arquivo de ícone não encontrado: {path}. Usando placeholder.")
                 img = Image.new("RGBA", resize_to or (50,50), (100, 100, 100, 150))
                 draw_placeholder = ImageDraw.Draw(img)
                 draw_placeholder.line([(0,0), img.size], fill=(255,0,0,200), width=2)
                 draw_placeholder.line([(0,img.height), (img.width,0)], fill=(255,0,0,200), width=2)
                 return img
        try:
            img = Image.open(path).convert("RGBA")
            if resize_to:
                img = img.resize(resize_to, RESAMPLE)
            return img
        except Exception as e:
            logger.error(f"Erro ao carregar ou redimensionar imagem '{name}': {e}")
            img = Image.new("RGBA", resize_to or (50,50), (100, 100, 100, 150))
            return img

    # --- Carrega o designer.png como BASE ---
    try:
         designer_base = load_img("designer.png", resize_to=(CANVAS_WIDTH, CANVAS_HEIGHT), is_essential=True)
    except FileNotFoundError:
         logger.critical("designer.png não encontrado. Não é possível gerar a imagem de perfil.")
         error_img = Image.new("RGB", (300, 100), "red")
         d = ImageDraw.Draw(error_img)
         d.text((10, 10), "Erro: designer.png não encontrado!", fill="white")
         buffer = io.BytesIO()
         error_img.save(buffer, format="PNG")
         buffer.seek(0)
         return buffer

    final_image = designer_base.copy()
    draw = ImageDraw.Draw(final_image)

    # --- Carrega Ícones ---
    icon_size = (50, 50)
    status_icon_size = (80, 80)
    insignia_size = (100, 100)
    flag_size = (75, 50)

    icon_instagram = load_img("icon_instagram.png", icon_size)
    icon_pronomes = load_img("icon_pronomes.png", icon_size)
    icon_casamento = load_img("icon_casamento.png", icon_size)

    status_icon = None
    status_name = user_profile.get("status")
    if status_name:
        status_map = {"Disponível": "status_disponivel.png", "Ausente": "status_ausente.png", "Ocupado": "status_ocupado.png"}
        if icon_filename := status_map.get(status_name):
             status_icon = load_img(icon_filename, status_icon_size)

    flag_img = None
    loc = user_profile.get("localizacao")
    if loc:
        flag_path = os.path.join(IMAGE_FOLDER, "bandeiras", f"{loc}.png")
        if os.path.exists(flag_path):
             try:
                 flag_img = Image.open(flag_path).convert("RGBA").resize(flag_size, RESAMPLE)
             except Exception as e:
                 logger.warning(f"Erro ao processar bandeira {loc}: {e}")

    insignia_images = []
    for insignia_filename in user_profile.get("insignias", [])[:3]:
        insignia_img = load_img(insignia_filename, insignia_size)
        insignia_images.append(insignia_img)

    # --- Avatar Redondo ---
    avatar_size = (470, 470)
    avatar_pos = (220, 148)    # Posição Avatar (X, Y FIXA)

    if pfp_data:
        try:
            pfp_image = Image.open(io.BytesIO(pfp_data)).convert("RGBA")
        except Exception: # Fallback se pfp_data for inválido
             pfp_image = Image.new("RGBA", avatar_size, (80, 80, 80, 255))
    else:
        pfp_image = Image.new("RGBA", avatar_size, (80, 80, 80, 255))

    pfp_image = pfp_image.resize(avatar_size, RESAMPLE)

    avatar_mask = Image.new("L", avatar_size, 0)
    mask_draw = ImageDraw.Draw(avatar_mask)
    mask_draw.ellipse((0, 0, avatar_size[0], avatar_size[1]), fill=255)

    final_image.paste(pfp_image, avatar_pos, avatar_mask)

    # ==============================================================
    # DESENHO DOS ELEMENTOS COM POSIÇÕES FIXAS
    # (Ajuste as coordenadas (X, Y) e cores aqui)
    # ==============================================================
    default_text_color = (220, 220, 220)

    # --- Nível e XP ---
    level_pos = (760, 960)
    bar_pos = (1000, 980)
    bar_width = 800
    bar_height = 40
    bar_radius = 20

    current_xp_raw = user_profile.get("xp", 0)
    if not isinstance(current_xp_raw, (int, float)) or current_xp_raw < 0:
         current_xp = 0
    else:
         current_xp = int(current_xp_raw)

    level = calculate_level(current_xp)
    xp_level_start = xp_for_level(level)
    xp_level_end = xp_for_level(level + 1)
    xp_in_level = max(0, current_xp - xp_level_start)
    xp_needed_for_next = max(1, xp_level_end - xp_level_start)
    progress_percentage = max(0, min(1, xp_in_level / xp_needed_for_next))

    draw.text(level_pos, f"Nível {level}", font=level_font, fill=default_text_color)

    draw.rounded_rectangle((bar_pos[0], bar_pos[1], bar_pos[0] + bar_width, bar_pos[1] + bar_height), radius=bar_radius, fill=(40, 40, 40, 200))
    if progress_percentage > 0:
        draw.rounded_rectangle((bar_pos[0], bar_pos[1], bar_pos[0] + int(bar_width * progress_percentage), bar_pos[1] + bar_height), radius=bar_radius, fill=(255, 255, 255, 255))

    xp_text = f"{xp_in_level} / {xp_needed_for_next} XP"
    xp_text_bbox = draw.textbbox((0, 0), xp_text, font=xp_font)
    xp_text_width = xp_text_bbox[2] - xp_text_bbox[0]
    xp_text_height = xp_text_bbox[3] - xp_text_bbox[1]
    xp_text_pos = (bar_pos[0] + (bar_width - xp_text_width) / 2, bar_pos[1] + (bar_height - xp_text_height) / 2 - 5)
    draw.text(xp_text_pos, xp_text, font=xp_font, fill=(10, 10, 10, 200))

    # --- Nome de Usuário ---
    username_pos = (720, 250)
    display_name = getattr(member, "display_name", "Usuário")
    draw.text((username_pos[0] + 2, username_pos[1] + 2), display_name, font=username_font, fill=(0, 0, 0, 180)) # Sombra
    draw.text(username_pos, display_name, font=username_font, fill=default_text_color)

    # --- Bandeira (Localização) ---
    if flag_img:
        username_bbox = draw.textbbox(username_pos, display_name, font=username_font)
        flag_pos = (username_bbox[2] + 20, username_pos[1] + (username_font.size - flag_size[1]) // 2)
        final_image.paste(flag_img, flag_pos, flag_img)

    # --- Bio ---
    bio_pos = (720, 400)
    bio_max_width = 1600 # Largura ajustada
    bio_text = user_profile.get("bio", "Bio não definida.")
    wrapped_bio = text_wrap(bio_text, bio_font, max_width=bio_max_width)
    draw.multiline_text(bio_pos, wrapped_bio, font=bio_font, fill=default_text_color, spacing=4)

    # --- Status ---
    if status_icon:
        status_pos = (100, 50)
        final_image.paste(status_icon, status_pos, status_icon)

    # --- Pronomes ---
    pronomes_text = user_profile.get("pronomes")
    if pronomes_text:
        pronoun_icon_pos = (100, 650)
        pronoun_text_pos = (pronoun_icon_pos[0] + icon_size[0] + 10, pronoun_icon_pos[1] + (icon_size[1] - info_font.size)//2)
        final_image.paste(icon_pronomes, pronoun_icon_pos, icon_pronomes)
        draw.text(pronoun_text_pos, pronomes_text, font=info_font, fill=default_text_color)

    # --- Instagram ---
    instagram_text = user_profile.get("instagram")
    if instagram_text:
        insta_icon_pos = (100, 720)
        insta_text_pos = (insta_icon_pos[0] + icon_size[0] + 10, insta_icon_pos[1] + (icon_size[1] - info_font.size)//2)
        final_image.paste(icon_instagram, insta_icon_pos, icon_instagram)
        draw.text(insta_text_pos, instagram_text, font=info_font, fill=default_text_color)

    # --- Casamento ---
    partner_name = user_profile.get("casado_com_nome")
    if partner_name:
        casamento_icon_pos = (100, 790)
        casado_text = f"Casado(a) com {partner_name}"
        casamento_text_pos = (casamento_icon_pos[0] + icon_size[0] + 10, casamento_icon_pos[1] + (icon_size[1] - info_font.size)//2)
        final_image.paste(icon_casamento, casamento_icon_pos, icon_casamento)
        draw.text(casamento_text_pos, casado_text, font=info_font, fill=default_text_color)

    # --- Insígnias ---
    insignia_y = CANVAS_HEIGHT - insignia_size[1] - 30
    insignia_spacing = 20
    total_insignia_width = len(insignia_images) * insignia_size[0] + max(0, len(insignia_images) - 1) * insignia_spacing
    start_x = CANVAS_WIDTH - total_insignia_width - 50 # Alinhado à direita

    current_x = start_x
    for insignia_img in insignia_images:
        insignia_pos = (current_x, insignia_y)
        final_image.paste(insignia_img, insignia_pos, insignia_img)
        current_x += insignia_size[0] + insignia_spacing

    # --- Salvar Imagem Final ---
    buffer = io.BytesIO()
    try:
        final_image.save(buffer, format="PNG", quality=95)
        buffer.seek(0)
        return buffer
    except Exception as e:
         logger.error(f"[Perfil {user_id_for_log}] Erro ao salvar imagem no buffer: {e}", exc_info=True)
         # Cria uma imagem de erro se falhar ao salvar
         error_img = Image.new("RGB", (300, 100), "red")
         d = ImageDraw.Draw(error_img)
         d.text((10, 10), "Erro ao salvar imagem!", fill="white")
         buffer = io.BytesIO()
         error_img.save(buffer, format="PNG")
         buffer.seek(0)
         return buffer

# --- Mod Log Helper ---
async def log_action(guild: discord.Guild, text: str):
    log_message = f"[MOD LOG] {guild.name} - {text}"; print(log_message)
    if MOD_LOG_CHANNEL_ID:
        log_channel = guild.get_channel(MOD_LOG_CHANNEL_ID)
        if log_channel and isinstance(log_channel, discord.TextChannel):
            try: await log_channel.send(text)
            except discord.Forbidden: print(f"ERRO: Sem permissão no canal de log {MOD_LOG_CHANNEL_ID}.")
            except Exception as e: print(f"ERRO: Falha ao enviar log: {e}")
        else: print(f"AVISO: Canal de log {MOD_LOG_CHANNEL_ID} não encontrado em '{guild.name}'.")

# --- Spotify OAuth URL Helper ---
def make_spotify_oauth_url(state: str) -> str:
    params = {"client_id": SPOTIFY_CLIENT_ID, "response_type": "code", "redirect_uri": REDIRECT_URI, "scope": " ".join(OAUTH_SCOPES), "state": state, "show_dialog": "true"}
    return "https://accounts.spotify.com/authorize?" + urllib.parse.urlencode(params)

# --- Emoji Info Helper ---
def extract_emoji_info(emoji_str):
    match = re.match(r"<(a?):(\w+):(\d+)>", emoji_str)
    return (int(match[3]), match[2], match[1] == 'a') if match else (None, None, False)


# ====================================================================================================
#   VIEWS (Unificadas)
# ====================================================================================================

# --- View de Login Spotify ---
class LoginView(View):
    def __init__(self, oauth_url: str, *, timeout: int = 600):
        super().__init__(timeout=timeout)
        self.add_item(Button(label="Login com Spotify", url=oauth_url, style=discord.ButtonStyle.link, emoji="🎵"))

# --- Views de Ticket ---
class Dropdown(discord.ui.Select):
    def __init__(self):
        options = [
            discord.SelectOption(value="boost",label="Seja Booster",emoji="🚀"),
            discord.SelectOption(value="classic",label="Aurora Classic",emoji="🩶"),
            discord.SelectOption(value="divine",label="Aurora Divine",emoji="💚"),
            discord.SelectOption(value="supreme",label="Aurora Supreme",emoji="💜"),
            discord.SelectOption(value="comprovante",label="Resgatar vantagens",emoji="💸")
        ]
        super().__init__(placeholder="Selecione uma opção...", min_values=1, max_values=1, options=options, custom_id="persistent_view:dropdown_help")

    # CORRIGIDO: Adicionado 'self'
    async def callback(self, interaction: discord.Interaction):
        value = self.values[0]
        if value == "boost": await interaction.response.send_message("""**Booster**\n> 💠 Cargo destacado\n> 🎨 Cores customizadas\n> ⚡ +100% XP\n> ✏️ Alterar apelido\n> 📎 Mídias e links\n> 💬 Chat Booster\n> 💎 Canal Pérolas\n⏳ Duração: Boost ativo\n*[🚀 impulsione]()*""", ephemeral=True)
        elif value == "classic": await interaction.response.send_message("""**Aurora Classic – R$10**\n> 💠 Cargo destacado\n> 🎨 Cores customizadas\n> 🏷️ Tag personalizada\n> 📞 Call personalizada\n> 💎 Canal Pérolas\n> ⚡ +50% XP\n⏳ Duração: 30 dias\n📜 Termos: após pagar, selecione `resgatar`.\n*[🔗 Adquirir](https://biolivre.com.br/borealapp)*""", ephemeral=True)
        elif value == "divine": await interaction.response.send_message("""**Aurora Divine – R$25**\n> 💠 Cargo destacado\n> 🎨 Cores customizadas\n> 🏷️ Tag personalizada\n> 📞 Call personalizada\n> 💎 Canal Pérolas\n> ⚡ +100% XP\n⏳ Duração: 30 dias\n📜 Termos: após pagar, selecione `resgatar`.\n*[🔗 Adquirir](https://biolivre.com.br/borealapp)*""", ephemeral=True)
        elif value == "supreme": await interaction.response.send_message("""**Aurora Supreme – R$80**\n> 💠 Cargo destacado\n> 🎨 Cores customizadas\n> 🏷️ Tag personalizada\n> 📞 Call personalizada\n> 💍 1x @Primeira Dama\n> 🎁 1x Aurora Classic p/ amigo\n> 🔇 Permissão mute\n> ⚡ +200% XP\n⏳ Duração: 30 dias\n📜 Termos: após pagar, selecione `resgatar`.\n*[🔗 Adquirir](https://biolivre.com.br/borealapp)*""", ephemeral=True)
        elif value == "comprovante": await interaction.response.send_message("> Crie um ticket no botão abaixo e envie o comprovante.", ephemeral=True, view=CreateTicket())

class DropdownView(discord.ui.View):
    def __init__(self):
        super().__init__(timeout=None)
        self.add_item(Dropdown())

class CreateTicket(discord.ui.View):
    def __init__(self):
        super().__init__(timeout=300)
        self.value=None

    @discord.ui.button(label="Resgatar VIP", style=discord.ButtonStyle.blurple, emoji="➕")
    # CORRIGIDO: Adicionado 'self'
    async def confirm(self, interaction: discord.Interaction, button: discord.ui.Button):
        self.value=True
        self.stop()
        ticket=None

        try:
             async for thread in interaction.channel.archived_threads(private=True, limit=None):
                  if f"({interaction.user.id})" in thread.name:
                       ticket = thread
                       break
             if not ticket:
                  for thread in interaction.channel.threads:
                       if isinstance(thread, discord.Thread) and thread.is_private() and f"({interaction.user.id})" in thread.name:
                            ticket = thread
                            break

             if ticket and not ticket.archived:
                  return await interaction.response.send_message(ephemeral=True, content=f"Você já tem um ticket aberto: {ticket.mention}")

             if ticket and ticket.archived:
                  await ticket.edit(archived=False, locked=False)
                  await ticket.edit(name=f"{interaction.user.display_name} ({interaction.user.id})", auto_archive_duration=10080) # invitable=False não é mais editável
                  await interaction.response.send_message(ephemeral=True, content=f"Seu ticket foi reaberto: {ticket.mention}")
                  await ticket.send(f"{interaction.user.mention} seu ticket foi reaberto!")
             else:
                  if not isinstance(interaction.channel, (discord.TextChannel, discord.ForumChannel)):
                       return await interaction.response.send_message("Não é possível criar tickets aqui.", ephemeral=True)

                  ticket = await interaction.channel.create_thread(
                       name=f"{interaction.user.display_name} ({interaction.user.id})",
                       auto_archive_duration=10080,
                       type=discord.ChannelType.private_thread
                  )
                  await interaction.response.send_message(ephemeral=True, content=f"Seu ticket foi criado: {ticket.mention}")
                  await ticket.send(f"""{interaction.user.mention} Bem-vindo! Envie o comprovante aqui. Pagamento: https://biolivre.com.br/borealapp
Use `/fecharticket` para encerrar.
<@&{ID_CARGO_ATENDENTE}>""")
        except discord.Forbidden:
             logger.error(f"Sem permissão para criar/gerenciar threads no canal {interaction.channel.id}")
             if not interaction.response.is_done():
                 await interaction.response.send_message("Erro: Não tenho permissão para criar ou gerenciar tickets neste canal.", ephemeral=True)
             else:
                 await interaction.followup.send("Erro: Não tenho permissão para criar ou gerenciar tickets neste canal.", ephemeral=True)
        except Exception as e:
             logger.exception(f"Erro ao criar/reabrir ticket para {interaction.user.id}:")
             if not interaction.response.is_done():
                 await interaction.response.send_message("Ocorreu um erro inesperado ao processar seu pedido de ticket.", ephemeral=True)
             else:
                 await interaction.followup.send("Ocorreu um erro inesperado ao processar seu pedido de ticket.", ephemeral=True)

# --- Views de Casamento ---
class ProposalView(ui.View):
    def __init__(self, proposer_id: int, target_id: int):
        super().__init__(timeout=300.0)
        self.proposer_id=proposer_id
        self.target_id=target_id
        self.message=None

    async def on_timeout(self):
        try:
            if self.message:
                for item in self.children: item.disabled = True
                await self.message.edit(content="💍 Proposta expirou.", view=self)
        except discord.NotFound:
            pass # Mensagem original foi deletada
        if self.target_id in client.pending_proposals:
            del client.pending_proposals[self.target_id]

    async def interaction_check(self, interaction: discord.Interaction) -> bool:
        if interaction.user.id != self.target_id:
            await interaction.response.send_message("Só o alvo pode responder.", ephemeral=True)
            return False
        return True

    @ui.button(label="Aceitar", style=discord.ButtonStyle.success, emoji="✅")
    async def accept_button(self, interaction: discord.Interaction, button: ui.Button):
        await interaction.response.defer()

        proposer = interaction.guild.get_member(self.proposer_id)
        target = interaction.user

        if not proposer:
             self.stop(); [setattr(item, 'disabled', True) for item in self.children]
             await interaction.followup.send(content="💔 Proponente saiu do servidor.", ephemeral=True)
             if self.message:
                 try: await self.message.edit(view=self)
                 except discord.NotFound: pass
             if self.target_id in client.pending_proposals: del client.pending_proposals[self.target_id]
             return

        try:
            # Pega ambos os perfis do MongoDB
            proposer_profile = await get_profile_bot(proposer.id)
            target_profile = await get_profile_bot(target.id)

            # Re-verifica se alguém casou
            if proposer_profile.get("casado_com_id") or target_profile.get("casado_com_id"):
                self.stop(); [setattr(item, 'disabled', True) for item in self.children]
                await interaction.followup.send(content="💔 Ops! Parece que um de vocês já está em um relacionamento.", ephemeral=True)
                if self.message:
                     try: await self.message.edit(view=self)
                     except discord.NotFound: pass
                if self.target_id in client.pending_proposals: del client.pending_proposals[self.target_id]
                return

            # Realiza o casamento (dois updates separados)
            await client.profiles.update_one(
                {"_id": str(proposer.id)},
                {"$set": {"casado_com_id": target.id, "casado_com_nome": target.display_name}}
            )
            await client.profiles.update_one(
                {"_id": str(target.id)},
                {"$set": {"casado_com_id": proposer.id, "casado_com_nome": proposer.display_name}}
            )

            # Limpa proposta pendente e desabilita botões
            if self.target_id in client.pending_proposals: del client.pending_proposals[self.target_id]
            self.stop(); [setattr(item, 'disabled', True) for item in self.children]

            if self.message:
                 try:
                     await self.message.edit(content=f"🎉 {proposer.mention} e {target.mention} agora estão casados!", view=self)
                 except discord.NotFound:
                     await interaction.followup.send(f"🎉 {proposer.mention} e {target.mention} agora estão casados!", ephemeral=False)
            else:
                 await interaction.followup.send(f"🎉 {proposer.mention} e {target.mention} agora estão casados!", ephemeral=False)

        except Exception as e:
             logger.error(f"Erro ao aceitar casamento (MongoDB): {e}", exc_info=True)
             try:
                 await interaction.followup.send("Ocorreu um erro ao processar a aceitação.", ephemeral=True)
             except discord.NotFound:
                 pass
             self.stop(); [setattr(item, 'disabled', True) for item in self.children]
             if self.message:
                 try: await self.message.edit(view=self)
                 except discord.NotFound: pass

    @ui.button(label="Recusar", style=discord.ButtonStyle.danger, emoji="❌")
    async def decline_button(self, interaction: discord.Interaction, button: ui.Button):
        proposer = interaction.guild.get_member(self.proposer_id)
        if self.target_id in client.pending_proposals: del client.pending_proposals[self.target_id]
        self.stop(); [setattr(item, 'disabled', True) for item in self.children]

        # Usa edit_message porque a interação é o clique no botão
        await interaction.response.edit_message(content=f"💔 Proposta de {proposer.mention if proposer else 'alguém'} recusada.", view=self)

# --- Views do Spotify ---
class VolumeControlView(View):
    def __init__(self, user_id: str, main_interaction: discord.Interaction, parent_view: 'PlaybackControlsView'):
        super().__init__(timeout=180)
        self.user_id = user_id
        self.main_interaction = main_interaction
        self.parent_view = parent_view

    async def interaction_check(self, i: discord.Interaction):
        if str(i.user.id) != self.user_id:
            await i.response.send_message("Controles não são para você.", ephemeral=True)
            return False
        return True

    async def _set_volume(self, i: discord.Interaction, volume: int):
        await i.response.defer()
        try:
            await spotify_request(client.http_session, self.user_id, "PUT", f"/me/player/volume?volume_percent={volume}")

            if self.parent_view:
                # Passa a interação 'i' do botão de volume para o _update_view
                await self.parent_view._update_view(i, update_volume_footer=True)

            # Deleta a mensagem ("Escolha volume:")
            await i.delete_original_response()
            self.stop()
        except (SpotifyAPIError, SpotifyAuthError) as e:
            await i.followup.send(f"Erro Spotify: {e}", ephemeral=True)
            self.stop()
        except Exception as e:
            logger.error(f"Erro set_volume: {e}", exc_info=True)
            await i.followup.send("Erro inesperado.", ephemeral=True)
            self.stop()

    @discord.ui.button(label="0%")
    async def v0(self,i,b): await self._set_volume(i,0)
    @discord.ui.button(label="25%")
    async def v25(self,i,b): await self._set_volume(i,25)
    @discord.ui.button(label="50%",style=discord.ButtonStyle.primary)
    async def v50(self,i,b): await self._set_volume(i,50)
    @discord.ui.button(label="75%")
    async def v75(self,i,b): await self._set_volume(i,75)
    @discord.ui.button(label="100%")
    async def v100(self,i,b): await self._set_volume(i,100)


class PlaybackControlsView(View):
    def __init__(self, user_id: str, player_state: Dict[str, Any], main_interaction: discord.Interaction):
        super().__init__(timeout=600)
        self.user_id = user_id
        self.player_state = player_state
        self.main_interaction = main_interaction

        discord.utils.get(self.children, custom_id="play_pause").emoji="⏸️" if player_state.get("is_playing") else "▶️"
        discord.utils.get(self.children, custom_id="toggle_shuffle").style=discord.ButtonStyle.success if player_state.get("shuffle_state") else discord.ButtonStyle.secondary
        repeat_button=discord.utils.get(self.children, custom_id="toggle_repeat"); repeat_state=player_state.get("repeat_state", "off")
        if repeat_state=="track": repeat_button.emoji, repeat_button.style = "🔂", discord.ButtonStyle.success
        elif repeat_state=="context": repeat_button.emoji, repeat_button.style = "🔁", discord.ButtonStyle.success
        else: repeat_button.emoji, repeat_button.style = "🔁", discord.ButtonStyle.secondary

    async def interaction_check(self, i: discord.Interaction):
        if str(i.user.id) != self.user_id:
            await i.response.send_message("Controles não são para você.", ephemeral=True)
            return False
        return True

    async def on_timeout(self):
        try:
            message = await self.main_interaction.original_response()
            for item in self.children: item.disabled = True
            await message.edit(view=self)
        except (discord.NotFound, discord.HTTPException):
            pass

    async def _update_view(self, interaction: discord.Interaction, update_volume_footer: bool = False):
        if not interaction.response.is_done():
            await interaction.response.defer()

        try:
            original_message = await self.main_interaction.original_response()

            if update_volume_footer:
                await asyncio.sleep(0.5)

            new_player_state = await spotify_request(client.http_session, self.user_id, "GET", "/me/player?market=from_token")

            if not new_player_state or not new_player_state.get("item"):
                await original_message.edit(content="Nada tocando.", embed=None, view=None, attachments=[])
                return

            self.player_state = new_player_state

            image_buffer = await create_spotify_embed_image(client.http_session, new_player_state)
            file = discord.File(image_buffer, filename="spotify_card.png")
            embed = discord.Embed(color=0x1DB954)
            embed.set_image(url="attachment://spotify_card.png")
            device = new_player_state.get('device', {})
            embed.set_footer(text=f"Tocando em {device.get('name', 'Spotify')} • Vol: {device.get('volume_percent', 'N/A')}%", icon_url="https://i.imgur.com/tG36kM6.png")

            view = PlaybackControlsView(user_id=self.user_id, player_state=new_player_state, main_interaction=self.main_interaction)

            await original_message.edit(content=None, embed=embed, view=view, attachments=[file])

        except (SpotifyAuthError, SpotifyAPIError) as e:
            # Se a interação veio de um botão de controle (não do volume), envia followup
            if not update_volume_footer:
                await interaction.followup.send(f"❌ Erro ao atualizar: {e}", ephemeral=True)
            else:
                logger.error(f"Erro ao atualizar view após mudança de volume: {e}")
                # Não envia followup aqui, pois a interação original era do botão de volume
        except Exception as e:
            logger.exception("Erro ao atualizar view Spotify.")
            if not update_volume_footer:
                await interaction.followup.send("Erro inesperado ao atualizar.", ephemeral=True)

    async def _handle_control_click(self, interaction: discord.Interaction, spotify_call: callable, *args):
        if not interaction.response.is_done():
            await interaction.response.defer()

        for item in self.children:
            item.disabled = True

        # Usa edit_original_response (da interação do botão)
        try:
             await interaction.edit_original_response(view=self)
        except discord.NotFound:
             await interaction.followup.send("A mensagem original desapareceu.", ephemeral=True)
             return

        try:
            await spotify_call(*args)
            await asyncio.sleep(0.75)
            await self._update_view(interaction)
        except (SpotifyAuthError, SpotifyAPIError) as e:
            logger.error(f"Erro no comando Spotify: {e}", exc_info=True)
            await interaction.followup.send(f"Erro Spotify: {e}", ephemeral=True)
            # Re-habilita botões se falhar
            for item in self.children:
                item.disabled = False
            try:
                await interaction.edit_original_response(view=self)
            except discord.NotFound:
                pass # Ignora se a mensagem sumiu

    @discord.ui.button(emoji="⏮️", custom_id="prev_track", row=0)
    async def prev(self, i: discord.Interaction, b: discord.ui.Button):
        await self._handle_control_click(i, spotify_request, client.http_session, self.user_id, "POST", "/me/player/previous")

    @discord.ui.button(emoji="▶️", custom_id="play_pause", row=0)
    async def play_pause(self, i: discord.Interaction, b: discord.ui.Button):
        endpoint = "/me/player/pause" if self.player_state.get("is_playing") else "/me/player/play"
        await self._handle_control_click(i, spotify_request, client.http_session, self.user_id, "PUT", endpoint)

    @discord.ui.button(emoji="⏭️", custom_id="next_track", row=0)
    async def next(self, i: discord.Interaction, b: discord.ui.Button):
        await self._handle_control_click(i, spotify_request, client.http_session, self.user_id, "POST", "/me/player/next")

    @discord.ui.button(emoji="🔊", custom_id="change_volume", row=1)
    async def volume(self, i: discord.Interaction, b: discord.ui.Button):
        view = VolumeControlView(self.user_id, self.main_interaction, parent_view=self)
        await i.response.send_message("Escolha volume:", view=view, ephemeral=True)

    @discord.ui.button(emoji="🔀", custom_id="toggle_shuffle", row=1)
    async def shuffle(self, i: discord.Interaction, b: discord.ui.Button):
        new_state = not self.player_state.get("shuffle_state", False)
        endpoint = f"/me/player/shuffle?state={str(new_state).lower()}"
        await self._handle_control_click(i, spotify_request, client.http_session, self.user_id, "PUT", endpoint)

    @discord.ui.button(emoji="🔁", custom_id="toggle_repeat", row=1)
    async def repeat(self, i: discord.Interaction, b: discord.ui.Button):
        cycle = {"off": "context", "context": "track", "track": "off"}
        new_state = cycle[self.player_state.get("repeat_state", "off")]
        endpoint = f"/me/player/repeat?state={new_state}"
        await self._handle_control_click(i, spotify_request, client.http_session, self.user_id, "PUT", endpoint)

    @discord.ui.button(emoji="🔗", custom_id="share_track", row=1)
    async def share(self, i: discord.Interaction, button: Button):
        await i.response.defer(ephemeral=True)
        track_url = self.player_state.get("item",{}).get("external_urls",{}).get("spotify")
        if not track_url:
             await i.followup.send("Não foi possível encontrar o link da música.", ephemeral=True)
             return

        try:
            image_buffer = await create_spotify_share_image(client.http_session, self.player_state, i.user.display_name)
            file = discord.File(image_buffer, filename="spotify_share.png")
            share_view = View()
            share_view.add_item(Button(label="Ouvir no Spotify", url=track_url, emoji="🎵"))
            await i.channel.send(f"Compartilhado por {i.user.mention}:", file=file, view=share_view)
            await i.followup.send("Música partilhada!", ephemeral=True)
        except Exception as e:
            logger.error(f"Falha partilhar Spotify: {e}", exc_info=True)
            await i.followup.send("Não foi possível partilhar a música.", ephemeral=True)

# --- Views de Moderação ---
class ConfirmView(View):
    def __init__(self, guild: discord.Guild, target: discord.Member, action: str, reason: Optional[str] = None, timeout: int = 60):
        super().__init__(timeout=timeout)
        self.guild = guild; self.target = target; self.action = action; self.reason = reason; self.result: Optional[str] = None
        logger.debug(f"ConfirmView: Ação '{self.action}' Alvo '{self.target}'.")

    @discord.ui.button(label="Confirmar", style=discord.ButtonStyle.danger)
    async def confirm(self, interaction: discord.Interaction, button: Button):
        if interaction.user.id != OWNER_ID:
            return await interaction.response.send_message("Não permitido.", ephemeral=True)
        await interaction.response.defer(ephemeral=True, thinking=True)

        try:
            if self.action == "disconnect":
                if self.target.voice and self.target.voice.channel:
                    await self.target.move_to(None, reason=self.reason)
                    self.result = f"✅ {self.target.display_name} desconectado."
                else:
                    self.result = f"⚠️ {self.target.display_name} não está em call."

            elif self.action == "ban":
                if interaction.guild.me.guild_permissions.ban_members:
                    await interaction.guild.ban(self.target, reason=self.reason)
                    self.result = f"✅ {self.target.display_name} banido."
                else:
                    self.result = "❌ Bot sem permissão para banir."

            elif self.action == "kick":
                if interaction.guild.me.guild_permissions.kick_members:
                    await self.target.kick(reason=self.reason)
                    self.result = f"✅ {self.target.display_name} expulso."
                else:
                    self.result = "❌ Bot sem permissão para expulsar."

            elif self.action == "mute_role":
                role = discord.utils.get(self.guild.roles, name=MUTED_ROLE_NAME)
                if role is None:
                    if interaction.guild.me.guild_permissions.manage_roles:
                        logger.info(f"Criando cargo '{MUTED_ROLE_NAME}'...")
                        role = await self.guild.create_role(name=MUTED_ROLE_NAME, reason="Painel Mod")
                        self.result = f"⚠️ Cargo '{MUTED_ROLE_NAME}' criado, ajuste permissões."
                    else:
                        self.result = f"❌ Cargo '{MUTED_ROLE_NAME}' não existe, bot sem permissão para criar."
                        await interaction.edit_original_response(content=self.result, view=None)
                        await log_action(self.guild, self.result)
                        self.stop()
                        return

                if interaction.guild.me.guild_permissions.manage_roles:
                    if role < interaction.guild.me.top_role:
                        await self.target.add_roles(role, reason=self.reason)
                        self.result = f"✅ {self.target.display_name} recebeu cargo '{role.name}'."
                    else:
                        self.result = f"❌ Não posso adicionar o cargo '{role.name}' (mais alto que o meu)."
                else:
                    self.result = "❌ Bot sem permissão para gerenciar cargos."

            elif self.action == "remove_roles":
                if interaction.guild.me.guild_permissions.manage_roles:
                    bot_top_role = interaction.guild.me.top_role
                    roles_to_remove = [r for r in self.target.roles if not r.is_default() and r < bot_top_role]
                    if roles_to_remove:
                        try:
                            await self.target.remove_roles(*roles_to_remove, reason=self.reason)
                            removed = [r.name for r in roles_to_remove]
                            self.result = f"✅ Cargos removidos de {self.target.display_name}: {', '.join(removed)}."
                        except Exception as e:
                             logger.error(f"Erro ao remover múltiplos cargos de '{self.target.name}': {e}")
                             self.result = f"⚠️ Erro ao remover alguns cargos de {self.target.display_name}."
                    else:
                         self.result = f"⚠️ Nenhum cargo elegível para remoção em {self.target.display_name}."
                else:
                    self.result = "❌ Bot sem permissão para gerenciar cargos."
            else:
                self.result = "❌ Ação desconhecida."

        except discord.Forbidden:
             self.result = f"❌ Sem permissão Discord para '{ACTION_NAMES.get(self.action, self.action)}'."
        except Exception as e:
            self.result = f"🚨 Falha crítica: {e}"
            logger.exception(f"Erro crítico /modpanel:")

        for item in self.children: item.disabled = True
        await interaction.edit_original_response(content=self.result, view=self)
        await log_action(self.guild, f"Ação: {ACTION_NAMES.get(self.action, self.action)} | Alvo: {self.target} ({self.target.id}) | Mod: {interaction.user} | Res: {self.result}")
        self.stop()

    @discord.ui.button(label="Cancelar", style=discord.ButtonStyle.secondary)
    async def cancel(self, interaction: discord.Interaction, button: Button):
        if interaction.user.id != OWNER_ID:
            return await interaction.response.send_message("Não permitido.", ephemeral=True)
        for item in self.children: item.disabled = True
        await interaction.response.edit_message(content="Ação cancelada.", view=self)
        self.result = "cancelado"
        self.stop()

class ModPanelView(View):
    def __init__(self):
        super().__init__(timeout=300)
        self.target: Optional[discord.Member] = None
        self.action: Optional[str] = None
        self._add_components()

    def _add_components(self):
        user_select = UserSelect(placeholder="1. Selecione o usuário alvo", min_values=1, max_values=1, row=0)
        user_select.callback = self.user_select_callback
        self.add_item(user_select)

        options = [discord.SelectOption(label=v, value=k, description=f"Ação: {v}") for k,v in ACTION_NAMES.items()]
        action_select = Select(placeholder='2. Escolha uma ação', min_values=1, max_values=1, options=options, row=1)
        action_select.callback = self.action_select_callback
        self.add_item(action_select)

    async def user_select_callback(self, interaction: discord.Interaction):
        user_id_str = interaction.data.get('values', [None])[0]
        if not user_id_str:
            return await interaction.response.send_message("Erro ao obter ID.", ephemeral=True)
        try:
             member = interaction.guild.get_member(int(user_id_str))
             if not member:
                 return await interaction.response.send_message("Membro não encontrado.", ephemeral=True)
             self.target = member
             logger.debug(f"ModPanel: Alvo definido {member.display_name}")
             await interaction.response.send_message(f"Alvo selecionado: **{member.display_name}**", ephemeral=True)
        except ValueError:
             await interaction.response.send_message("ID de usuário inválido.", ephemeral=True)
        except Exception as e:
             logger.error(f"Erro user_select_callback: {e}")
             await interaction.response.send_message("Erro ao processar seleção.", ephemeral=True)

    async def action_select_callback(self, interaction: discord.Interaction):
        if interaction.user.id != OWNER_ID:
            return await interaction.response.send_message("Apenas dono.", ephemeral=True)

        self.action = interaction.data.get('values', [None])[0]
        if not self.action:
            return await interaction.response.send_message("Nenhuma ação.", ephemeral=True)
        if not self.target:
            return await interaction.response.send_message("Selecione usuário primeiro.", ephemeral=True)

        if self.target.id == OWNER_ID:
            return await interaction.response.send_message("Não pode aplicar ações no Dono.", ephemeral=True)
        if self.target.id == interaction.client.user.id:
            return await interaction.response.send_message("Não posso aplicar ações em mim mesmo.", ephemeral=True)

        if self.target.top_role >= interaction.guild.me.top_role:
             return await interaction.response.send_message(f"Não posso aplicar ações em {self.target.display_name} (cargo mais alto).", ephemeral=True)

        embed = discord.Embed(title="⚠️ Confirmação", description="Confirme a ação de moderação.", color=discord.Color.orange())
        embed.add_field(name="Alvo", value=f"`{self.target.display_name}` ({self.target.id})", inline=False)
        embed.add_field(name="Ação", value=f"`{ACTION_NAMES.get(self.action, self.action)}`", inline=False)
        embed.set_footer(text="Expira em 60s.")
        confirm_view = ConfirmView(interaction.guild, self.target, self.action, reason=f"Ação via painel por {interaction.user.display_name}")
        await interaction.response.send_message(embed=embed, view=confirm_view, ephemeral=True)

class ConfirmationViewPainel(discord.ui.View):
    def __init__(self, action: str, member: discord.Member, original_view: discord.ui.View):
        super().__init__(timeout=30)
        self.action = action; self.member = member; self.original_view = original_view
        self.message = None

    async def disable_original_buttons(self):
        for item in self.original_view.children:
            if isinstance(item, (discord.ui.Button, discord.ui.Select)): # Desabilita selects também
                item.disabled = True

        original_message = getattr(self.original_view, 'message', None)

        if original_message and isinstance(original_message, discord.Message):
            try:
                await original_message.edit(view=self.original_view)
            except (discord.NotFound, discord.HTTPException):
                pass

    @discord.ui.button(label="Confirmar", style=discord.ButtonStyle.danger)
    async def confirm(self, interaction: discord.Interaction, button: discord.ui.Button):
        if not interaction.user.guild_permissions.administrator:
            return await interaction.response.send_message("Sem permissão admin.", ephemeral=True)

        await interaction.response.defer(ephemeral=True)

        # Re-verifica hierarquia no momento do clique
        if self.member.top_role >= interaction.guild.me.top_role:
            msg = f"❌ Não posso aplicar '{self.action}' em {self.member.mention} (cargo mais alto)."
            view_to_send = None
        else:
            try:
                if self.action == "ban":
                    if interaction.guild.me.guild_permissions.ban_members:
                         await self.member.ban(reason=f"Painel por {interaction.user.name}")
                         msg = f"✅ {self.member.mention} banido por {interaction.user.mention}!"
                    else:
                         msg = "❌ Bot sem permissão 'Banir Membros'."

                elif self.action == "kick":
                    if interaction.guild.me.guild_permissions.kick_members:
                        await self.member.kick(reason=f"Painel por {interaction.user.name}")
                        msg = f"✅ {self.member.mention} expulso por {interaction.user.mention}!"
                    else:
                        msg = "❌ Bot sem permissão 'Expulsar Membros'."

                elif self.action == "mute":
                    if not self.member.voice or not self.member.voice.channel:
                        msg = f"⚠️ {self.member.mention} não está em um canal de voz."
                    elif interaction.guild.me.guild_permissions.mute_members:
                        await self.member.edit(mute=True, reason=f"Painel por {interaction.user.name}")
                        msg = f"✅ {self.member.mention} silenciado em calls por {interaction.user.mention}!"
                    else:
                        msg = "❌ Bot sem permissão 'Silenciar Membros'."

                else:
                    msg="❌ Ação inválida."
                    await interaction.edit_original_response(content=msg, view=None)
                    return

                view_to_send = None
                await self.disable_original_buttons()

            except discord.Forbidden:
                msg = f"❌ Sem permissão Discord para '{self.action}'."
                view_to_send = None
            except Exception as e:
                msg = f"❌ Erro: {e}"
                view_to_send = None
                logger.error(f"Erro ConfirmationViewPainel: {e}", exc_info=True)

        await interaction.edit_original_response(content=msg, view=view_to_send)

    @discord.ui.button(label="Cancelar", style=discord.ButtonStyle.secondary)
    async def cancel(self, interaction: discord.Interaction, button: discord.ui.Button):
        for item in self.children:
            item.disabled = True
        await interaction.response.edit_message(content="❌ Ação cancelada.", view=self)

class ModerationView(discord.ui.View):
    def __init__(self, member: discord.Member, interaction: discord.Interaction):
        super().__init__(timeout=180)
        self.member = member
        self.interaction = interaction
        self.message: Optional[discord.Message] = None

        bot_top_role = interaction.guild.me.top_role

        # Pega roles atuais ELEGÍVEIS para pré-seleção
        default_roles_objects = []
        try:
            default_roles_objects = [r for r in member.roles if not r.is_default() and r < bot_top_role]
            self.selected_roles = default_roles_objects # Salva os objetos
        except Exception:
             self.selected_roles = []
             logger.warning("Não foi possível pré-selecionar cargos para o RoleSelect.")

        # Timeout select
        self.timeout_select=discord.ui.Select(placeholder="⏳ Timeout (castigo)", options=[discord.SelectOption(label="1 hora",value="3600"), discord.SelectOption(label="6h",value="21600"), discord.SelectOption(label="1 dia",value="86400"), discord.SelectOption(label="1 sem",value="604800"), discord.SelectOption(label="Remover",value="0")])
        self.timeout_select.callback=self.timeout_callback
        self.add_item(self.timeout_select)

        # Role select
        # Tenta pré-selecionar marcando 'default=True' nas opções
        role_options = []
        all_eligible_roles = [r for r in interaction.guild.roles if not r.is_default() and r < bot_top_role and not r.managed]

        # Limita o número de opções para 25 (limite do Discord)
        # Damos prioridade aos cargos que o usuário JÁ TEM
        selected_role_ids = {r.id for r in self.selected_roles}
        options_count = 0

        # 1. Adiciona os cargos que o usuário já tem
        for role in self.selected_roles:
            if options_count < 25:
                role_options.append(discord.SelectOption(label=role.name, value=str(role.id), default=True))
                options_count += 1
            else:
                 break # Para se atingir 25

        # 2. Adiciona os cargos restantes (que o usuário não tem)
        if options_count < 25:
             for role in all_eligible_roles:
                 if role.id not in selected_role_ids: # Só adiciona se ainda não estiver na lista
                     role_options.append(discord.SelectOption(label=role.name, value=str(role.id), default=False))
                     options_count += 1
                     if options_count >= 25:
                         break # Para se atingir 25

        if not role_options:
             # Se não houver cargos elegíveis, adiciona um placeholder desabilitado
             role_options.append(discord.SelectOption(label="Nenhum cargo gerenciável", value="disabled", default=False))

        self.role_select=discord.ui.RoleSelect(
            placeholder="🎖️ Gerenciar cargos",
            min_values=0,
            max_values=min(25, len(role_options)),
            # default_values=self.selected_roles # <-- Isto causa erro em versões antigas
            options=role_options # <-- Alternativa: passar opções pré-marcadas
        )
        self.role_select.callback=self.select_roles_callback
        self.add_item(self.role_select)

        self.add_item(discord.ui.Button(label="Banir", style=discord.ButtonStyle.danger, custom_id="ban_btn", row=2))
        self.add_item(discord.ui.Button(label="Expulsar", style=discord.ButtonStyle.danger, custom_id="kick_btn", row=2))
        self.add_item(discord.ui.Button(label="Silenciar Voz", style=discord.ButtonStyle.danger, custom_id="mute_btn", row=2))

        self.check_permissions()

    def check_permissions(self):
        try:
            bot_perms = self.interaction.app_permissions
        except Exception:
            bot_perms = self.interaction.guild.me.guild_permissions

        member_top_role = self.member.top_role
        bot_top_role = self.interaction.guild.me.top_role
        is_higher = bot_top_role > member_top_role

        self.timeout_select.disabled = not bot_perms.moderate_members or not is_higher

        # Desabilita RoleSelect se não houver opções ou permissão
        if not bot_perms.manage_roles or not self.role_select.options or self.role_select.options[0].value == "disabled":
             self.role_select.disabled = True

        ban_btn = discord.utils.get(self.children, custom_id="ban_btn")
        kick_btn = discord.utils.get(self.children, custom_id="kick_btn")
        mute_btn = discord.utils.get(self.children, custom_id="mute_btn")

        if ban_btn: ban_btn.disabled = not bot_perms.ban_members or not is_higher
        if kick_btn: kick_btn.disabled = not bot_perms.kick_members or not is_higher
        if mute_btn: mute_btn.disabled = not bot_perms.mute_members

    async def timeout_callback(self, interaction: discord.Interaction):
        if not interaction.user.guild_permissions.moderate_members:
            return await interaction.response.send_message("❌ Sem permissão 'Moderar Membros'.", ephemeral=True)
        if self.member.top_role >= interaction.guild.me.top_role:
            return await interaction.response.send_message("❌ Não posso aplicar timeout (cargo mais alto).", ephemeral=True)

        secs=int(interaction.data['values'][0])

        try:
            if secs > 0:
                await self.member.timeout(timedelta(seconds=secs), reason=f"/painel por {interaction.user.name}")
                await interaction.response.send_message(f"⏳ {self.member.mention} timeout por {self.format_duration(secs)}.", ephemeral=True)
            else:
                await self.member.timeout(None, reason=f"/painel por {interaction.user.name}")
                await interaction.response.send_message(f"✅ Timeout removido de {self.member.mention}.", ephemeral=True)
        except discord.Forbidden:
            await interaction.response.send_message("❌ Bot sem permissão 'Moderar Membros'.", ephemeral=True)
        except Exception as e:
            logger.error(f"Erro timeout_callback: {e}");
            await interaction.response.send_message(f"❌ Erro: {e}", ephemeral=True)

    def format_duration(self, seconds: int) -> str:
        periods=[('sem',604800),('d',86400),('h',3600),('m',60),('s',1)]; parts=[]
        for name, secs in periods:
            if seconds>=secs:
                val,seconds=divmod(seconds,secs)
                parts.append(f"{val}{name}")
        return ", ".join(parts) if parts else "0s"

    async def select_roles_callback(self, interaction: discord.Interaction):
        if not interaction.user.guild_permissions.manage_roles:
            return await interaction.response.send_message("❌ Sem permissão 'Gerenciar Cargos'.", ephemeral=True)

        old_roles = set(self.selected_roles) # Usa os objetos Role salvos

        # Pega os IDs selecionados na interação
        selected_role_ids_str = interaction.data.get('values', [])

        # Converte os IDs de string em objetos discord.Role
        new_roles = set()
        for role_id_str in selected_role_ids_str:
            try:
                role = interaction.guild.get_role(int(role_id_str))
                if role:
                    new_roles.add(role)
            except (ValueError, TypeError):
                pass

        # Salva os novos objetos Role
        self.selected_roles = list(new_roles)

        added = new_roles - old_roles
        removed = old_roles - new_roles
        bot_top_role = interaction.guild.me.top_role
        response = []

        # Filtra roles que o bot não pode gerenciar (redundante se a lista de opções foi bem feita, mas seguro)
        added_manageable = {r for r in added if r < bot_top_role}
        removed_manageable = {r for r in removed if r < bot_top_role}
        added_unmanageable = added - added_manageable
        removed_unmanageable = removed - removed_manageable

        reason=f"/painel por {interaction.user.name}"
        try:
            if added_manageable:
                await self.member.add_roles(*added_manageable, reason=reason)
                response.append(f"**➕ Add:** {', '.join(r.mention for r in added_manageable)}")
            if removed_manageable:
                await self.member.remove_roles(*removed_manageable, reason=reason)
                response.append(f"**➖ Remov:** {', '.join(r.mention for r in removed_manageable)}")

            if added_unmanageable: response.append(f"⚠️ **Não Add:** {', '.join(r.mention for r in added_unmanageable)} (Hierarquia)")
            if removed_unmanageable: response.append(f"⚠️ **Não Remov:** {', '.join(r.mention for r in removed_unmanageable)} (Hierarquia)")
            if not response: response.append("ℹ️ Nenhum cargo alterado.")

            await interaction.response.send_message("\n".join(response), ephemeral=True)

        except discord.Forbidden:
            await interaction.response.send_message("❌ Bot sem permissão 'Gerenciar Cargos'.", ephemeral=True)
        except Exception as e:
            logger.error(f"Erro select_roles_callback: {e}", exc_info=True)
            await interaction.response.send_message(f"❌ Erro: {e}", ephemeral=True)

    async def on_interaction(self, interaction: discord.Interaction):
        if interaction.type != discord.InteractionType.component:
            return

        custom_id = interaction.data.get('custom_id')
        if not custom_id:
            return

        action_map = {"ban_btn": "ban", "kick_btn": "kick", "mute_btn": "mute"}
        if custom_id in action_map:
            action = action_map[custom_id]
            perm_map = {"ban": "ban_members", "kick": "kick_members", "mute": "mute_members"}

            if not getattr(interaction.user.guild_permissions, perm_map[action], False):
                return await interaction.response.send_message(f"❌ Você não tem permissão '{perm_map[action]}'.", ephemeral=True)

            confirm_view = ConfirmationViewPainel(action, self.member, self)
            await interaction.response.send_message(f"⚠️ Confirmar **{action}** de {self.member.mention}?", view=confirm_view, ephemeral=True)

    async def on_timeout(self):
        for item in self.children: item.disabled = True
        if self.message:
            try: await self.message.edit(view=self)
            except (discord.NotFound, discord.HTTPException): pass
# ====================================================================================================
#   CLASSE PRINCIPAL DO BOT
# ====================================================================================================
class MyBot(commands.Bot):
    def __init__(self):
        intents = discord.Intents.default()
        intents.messages = True; intents.guilds = True; intents.members = True
        intents.message_content = True; intents.voice_states = True
        super().__init__(command_prefix="!", intents=intents, application_id=APP_ID)

        self.version = "3.0.0 (MongoDB Migration)"
        self.http_session: Optional[aiohttp.ClientSession] = None
        self.web_runner: Optional[web.AppRunner] = None
        self.xp_cooldowns: Dict[str, float] = {}
        self.pending_proposals: Dict[int, int] = {}

        # --- Conexão MongoDB ---
        # Pega as variáveis importadas do mongo_utils
        self.mongo_client = async_client
        self.db = db_async
        self.profiles = profiles_async # Esta é a coleção (ex: db["profiles"])

        if self.profiles is None: # <-- Corrigido para 'is None'
             logger.critical("FALHA AO CONECTAR AO MONGODB (Coleção 'profiles' é None). O Bot não pode iniciar.")
             # Você pode descomentar a linha abaixo para impedir o bot de rodar sem DB
             # raise RuntimeError("Não foi possível conectar ao MongoDB. Verifique 'mongo_utils.py' e a string de conexão.")
        else:
             logger.info("Referência da coleção MongoDB 'profiles' carregada.")
        # --- Fim da Conexão ---

    async def setup_hook(self):
        logger.info("Executando setup_hook...")

        # 1. Adiciona Views Persistentes
        self.add_view(DropdownView()) # View do Ticket
        self.add_view(DailyButtonView()) # View do Daily (Economia)
        logger.info("Views persistentes adicionadas.")

        # 2. Testa Conexão MongoDB Async
        if self.mongo_client:
            try:
                await self.mongo_client.server_info() # Testa a conexão
                logger.info("✅ Conexão MongoDB Async (Bot) BEM SUCEDIDA.")
            except Exception as e:
                logger.critical(f"❌ FALHA na conexão MongoDB Async no setup_hook: {e}")
        else:
             logger.warning("Cliente MongoDB (async_client) não foi inicializado.")

        # 3. Cria Sessão HTTP
        try:
            self.http_session = aiohttp.ClientSession()
            logger.info("Sessão aiohttp.ClientSession criada.")
        except Exception as e:
             logger.critical(f"Falha ao criar aiohttp.ClientSession: {e}", exc_info=True)
             self.http_session = None # Garante que está None se falhar

        # 4. Inicia Servidor Web (APENAS para Spotify)
        await self.start_web_server()

        # 5. Sincroniza Comandos
        try:
            if GUILD_ID_OBJ:
                 # Sincroniza apenas para o servidor de teste (mais rápido)
                 self.tree.copy_global_to(guild=GUILD_ID_OBJ)
                 synced = await self.tree.sync(guild=GUILD_ID_OBJ)
                 logger.info(f"Comandos registrados no servidor {GUILD_ID_PRINCIPAL} ({len(synced)} comandos)")
            else:
                 # Sincroniza globalmente (pode levar 1 hora para atualizar)
                 synced = await self.tree.sync()
                 logger.info(f"Comandos registrados globalmente ({len(synced)} comandos)")
        except Exception as e:
            logger.error(f"Erro ao sincronizar comandos: {e}", exc_info=True)

        logger.info("Setup hook concluído.")

    async def on_ready(self):
        await self.change_presence(activity=discord.CustomActivity(name="Use /help"))
        session_status = "Válida e aberta" if self.http_session and not self.http_session.closed else "INVÁLIDA ou fechada"
        logger.info(f"on_ready: Status da sessão HTTP: {session_status}")
        logger.info("="*30 + f"\nBot pronto. Logado como {self.user} (ID: {self.user.id})\nConectado a {len(self.guilds)} servidor(es).\nID Dono: {OWNER_ID}\nCanal Log: {MOD_LOG_CHANNEL_ID}\n" + "="*30)

    async def on_message(self, message: discord.Message):
        if not message.guild or message.author.bot: return

        # XP (Apenas no servidor principal)
        if str(message.guild.id) == GUILD_ID_PRINCIPAL:
            await self.process_xp(message)

        # Resposta IA
        await self.process_ia_reply(message)

        # Processa comandos ! (se houver)
        await self.process_commands(message)

    async def process_xp(self, message: discord.Message):
        author_id = str(message.author.id)
        current_time = time.time()

        if author_id not in self.xp_cooldowns or (current_time - self.xp_cooldowns.get(author_id, 0) >= 60):
            self.xp_cooldowns[author_id] = current_time
            xp_to_add = random.randint(15, 25)

            try:
                # 1. Pega o perfil (cria/migra se necessário)
                profile = await get_profile_bot(author_id)

                # 2. Pega o XP antigo
                old_xp = profile.get("xp", 0)
                new_xp = old_xp + xp_to_add

                # 3. Atualiza o XP no banco de dados
                await self.profiles.update_one(
                    {"_id": author_id},
                    {"$set": {"xp": new_xp}}
                )

                # 4. Lógica de Level Up
                old_level = calculate_level(old_xp)
                new_level = calculate_level(new_xp)

                if new_level > old_level:
                    try:
                        await message.channel.send(f"🎉 Parabéns, {message.author.mention}! Nível {new_level}!")
                    except discord.Forbidden:
                        pass

            except Exception as e:
                 logger.error(f"Erro ao processar XP (MongoDB) para {author_id}: {e}", exc_info=True)


    async def process_ia_reply(self, message: discord.Message):
        if not message.reference: return
        try:
            ref_msg = await message.channel.fetch_message(message.reference.message_id)
            if ref_msg.author == self.user and ("</ia:" in ref_msg.content or "Boreal IA" in ref_msg.content):
                async with message.channel.typing():
                    response = await model.generate_content_async(message.content)
                    resp_text = response.text[:1997]+"..." if len(response.text)>2000 else response.text
                    await message.reply(f"<:j_cerebrobot:1363237071011057725> **Boreal IA**\n{resp_text}")
        except discord.NotFound: pass
        except Exception as e: logger.warning(f"Erro reply IA: {e}")

    @commands.Cog.listener()
    async def on_member_ban(self, guild: discord.Guild, user: discord.User):
        if str(guild.id) != GUILD_ID_PRINCIPAL: return
        try:
            ban_entry = await guild.fetch_ban(user)
            motivo = ban_entry.reason or "N/A."
        except:
            motivo = "N/A."

        embed = discord.Embed(title="🚫 Banido(a)", color=0x2F3136, timestamp=discord.utils.utcnow())
        embed.add_field(name="Servidor", value=f"`{guild.name}`", inline=False)
        embed.add_field(name="Motivo", value=f"`{motivo}`", inline=False)
        embed.add_field(name="Duração", value="`Permanente`", inline=False)
        try:
            await user.send(embed=embed)
        except:
            pass # DM fechada

    async def close(self):
        logger.info("Fechando bot...")
        if self.http_session and not self.http_session.closed:
            logger.info("Fechando sessão aiohttp...")
            await self.http_session.close()
            logger.info("Sessão aiohttp fechada.")
        else:
            logger.info("Nenhuma sessão aiohttp ativa para fechar.")

        if self.web_runner:
            logger.info("Limpando web runner...")
            await self.web_runner.cleanup()
            logger.info("Web runner limpo.")

        await super().close()
        logger.info("Bot fechado.")

    # --- Métodos Webserver (APENAS Spotify) ---
    async def start_web_server(self):
        app = web.Application()
        # Adiciona APENAS as rotas que o BOT gerencia (Spotify)
        app.add_routes([
          web.get("/callback", self.handle_callback_spotify),
        ])

        self.web_runner = web.AppRunner(app)
        await self.web_runner.setup()
        site = web.TCPSite(self.web_runner, WEB_SERVER_HOST, WEB_SERVER_PORT)
        try:
            await site.start()
            logger.info(f"Webserver (AIOHTTP p/ Spotify) rodando: http://{WEB_SERVER_HOST}:{WEB_SERVER_PORT}")
        except OSError as e:
             logger.error(f"Falha ao iniciar webserver na porta {WEB_SERVER_PORT}: {e}")
             logger.warning("Continuando sem funcionalidade de callback do Spotify.")


    async def handle_callback_spotify(self, request: web.Request):
        params = request.rel_url.query
        code, state_id = params.get("code"), params.get("state")
        if not code or not state_id:
            return web.Response(text="Parâmetros ausentes.", status=400)

        # Verifica se a http_session existe
        if not self.http_session or self.http_session.closed:
             logger.error("Callback do Spotify falhou: http_session está fechada ou é None.")
             return web.Response(text="Erro interno do Bot (Sessão HTTP não iniciada).", status=500)

        try:
            async with self.http_session.post(
                "https://accounts.spotify.com/api/token",
                auth=aiohttp.BasicAuth(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET),
                data={"grant_type": "authorization_code", "code": code, "redirect_uri": REDIRECT_URI}
            ) as resp:
                resp.raise_for_status()
                tokens = await resp.json()

            await save_tokens_for_user(state_id, tokens)
            html = """<html><head><title>Login OK</title><style>body{font-family:Arial;background:#121212;color:#fff;display:flex;justify-content:center;align-items:center;height:100vh;margin:0}.container{text-align:center;padding:40px;background:#282828;border-radius:10px}h2{color:#1DB954}</style></head><body><div class="container"><h2>✅ Login Spotify OK!</h2><p>Pode fechar e voltar ao Discord.</p></div></body></html>"""
            return web.Response(text=html, content_type="text/html")
        except Exception as e:
            logger.exception("Erro callback OAuth (Spotify).");
            return web.Response(text=f"Erro login: {e}", status=500)

# ====================================================================================================
#   INSTÂNCIA DO BOT
# ====================================================================================================
client = MyBot()

# ====================================================================================================
#   COMANDOS SLASH (Definidos no escopo global, FORA da classe)
# ====================================================================================================

# --- Comandos (Tickets) ---
@client.tree.command(name='setup', description='TICKET | Configura o painel de tickets.')
@app_commands.guilds(GUILD_ID_OBJ) # Restringe ao servidor principal
@app_commands.checks.has_permissions(manage_guild=True)
async def setup(interaction: discord.Interaction): # <-- SEM SELF
    await interaction.response.send_message("Painel Criado", ephemeral=True)
    embed = discord.Embed(title='<:suporte_discord:1278496261330309123> CENTRO DE AJUDA', colour=0x979c9f)
    embed.description = ("Seja bem-vindo(a)!\n\nSelecione abaixo para suporte:\n\n"
                         "• **Seja Booster:** Vantagens.\n• **Aurora Classic:** VIP básico.\n"
                         "• **Aurora Divine:** VIP intermediário.\n• **Aurora Supreme:** VIP máximo.\n"
                         "• **Resgatar vantagens:** Já pagou? Resgate aqui.\n\n*Abuso resultará em punição.*")
    embed.set_image(url='https://media.discordapp.net/attachments/1309248111809396776/1311777808485322762/wC1juCB.png?ex=688922c9&is=6887d149&hm=0ae78ff813ba14f895887894feb6d53bcba131f0da666147adc7cde96e4ed614&')
    await interaction.channel.send(embed=embed, view=DropdownView())

@setup.error
async def setup_error(i: discord.Interaction, e: app_commands.AppCommandError):
    if isinstance(e, app_commands.MissingPermissions):
        await i.response.send_message("Sem permissão 'Gerenciar Servidor'.", ephemeral=True)
    else:
        logger.error(f"Erro /setup: {e}", exc_info=True)
        await i.response.send_message(f"Erro: {e}", ephemeral=True)

@client.tree.command(name="fecharticket", description='TICKET | Fecha um atendimento atual.')
@app_commands.guilds(GUILD_ID_OBJ)
async def _fecharticket(interaction: discord.Interaction): # <-- SEM SELF
    if not isinstance(interaction.channel, discord.Thread):
        return await interaction.response.send_message("Não é um ticket.", ephemeral=True)

    mod_role = interaction.guild.get_role(int(ID_CARGO_ATENDENTE))
    is_creator = f"({interaction.user.id})" in interaction.channel.name
    is_mod = mod_role and mod_role in interaction.user.roles

    if is_creator or is_mod:
        try:
             await interaction.response.send_message(f"Ticket arquivado por {interaction.user.mention}.")
             await interaction.channel.edit(archived=True, locked=True)
        except discord.Forbidden:
             await interaction.followup.send("Erro: Não tenho permissão para arquivar/trancar esta thread.", ephemeral=True)
        except Exception as e:
             logger.error(f"Erro ao fechar ticket {interaction.channel.id}: {e}", exc_info=True)
             await interaction.followup.send("Ocorreu um erro ao fechar o ticket.", ephemeral=True)
    else:
        await interaction.response.send_message("Você não pode fechar este ticket.", ephemeral=True)

# --- Comandos (Perfil & Casamento) ---
@client.tree.command(name="perfil", description="Mostra o seu perfil ou o de outro usuário.")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.describe(usuario="O usuário que você quer ver o perfil (opcional).")
async def perfil(interaction: discord.Interaction, usuario: discord.Member = None): # <-- SEM SELF
    await interaction.response.defer()
    target_user = usuario or interaction.user

    try:
        # 1. Pega o perfil do DB
        profile_data = await get_profile_bot(target_user.id)

        # 2. Baixa o avatar
        pfp_data = await fetch_image(client.http_session, target_user.display_avatar.url if target_user.display_avatar else None)

        # 3. Roda a geração da imagem (síncrona) em uma thread separada
        image_buffer = await asyncio.to_thread(
            generate_profile_image_sync,
            target_user, # Passa o objeto Member
            pfp_data,    # Passa os bytes do avatar
            profile_data # Passa o dicionário do perfil vindo do MongoDB
        )

        file = discord.File(fp=image_buffer, filename="perfil.png")
        await interaction.followup.send(file=file)

    except FileNotFoundError as e:
        logger.error(f"/perfil: {e}", exc_info=True)
        await interaction.followup.send(f"Erro: Arquivo essencial não encontrado ({e}).", ephemeral=True)
    except Exception as e:
        logger.exception(f"/perfil erro para {target_user.id}:")
        await interaction.followup.send("Erro ao gerar perfil.", ephemeral=True)

@client.tree.command(name="perfilbio", description="Define a sua biografia no perfil.")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.describe(bio="O texto da sua nova biografia (máx 150 caracteres).")
async def perfilbio(interaction: discord.Interaction, bio: str): # <-- SEM SELF
    if len(bio) > 150:
        return await interaction.response.send_message("Bio muito longa! Máximo de 150 caracteres.", ephemeral=True)

    user_id_str = str(interaction.user.id)
    try:
        # Atualiza a bio diretamente no MongoDB
        await client.profiles.update_one(
            {"_id": user_id_str},
            {"$set": {"bio": bio}},
            upsert=True # Cria o perfil se não existir
        )
        await interaction.response.send_message("✅ Bio atualizada com sucesso!", ephemeral=True)

    except Exception as e:
         logger.error(f"Erro ao salvar bio (MongoDB) para {user_id_str}: {e}", exc_info=True)
         await interaction.response.send_message("❌ Ocorreu um erro ao tentar salvar sua bio.", ephemeral=True)

# Comando /perfiledit (antigo) removido, substituído pelo /editarperfilui

@client.tree.command(name="casar", description="Peça alguém em casamento.")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.describe(usuario="A pessoa que quer pedir em casamento.")
async def casar(interaction: discord.Interaction, usuario: discord.Member): # <-- SEM SELF
    proposer = interaction.user
    target = usuario

    if proposer.id == target.id:
        return await interaction.response.send_message("Você não pode casar consigo mesmo!", ephemeral=True)
    if target.bot:
        return await interaction.response.send_message("Não pode casar com bots!", ephemeral=True)

    try:
        # Pega ambos os perfis do MongoDB
        proposer_profile = await get_profile_bot(proposer.id)
        target_profile = await get_profile_bot(target.id)

        # Verifica se o proponente já está casado
        proposer_partner_id = proposer_profile.get("casado_com_id")
        if proposer_partner_id is not None:
            partner_name = proposer_profile.get("casado_com_nome", f"<@{proposer_partner_id}>")
            return await interaction.response.send_message(f"Você já está casado(a) com {partner_name}!", ephemeral=True)

        # Verifica se o alvo já está casado
        target_partner_id = target_profile.get("casado_com_id")
        if target_partner_id is not None:
             partner_name = target_profile.get("casado_com_nome", f"<@{target_partner_id}>")
             return await interaction.response.send_message(f"{target.display_name} já está casado(a) com {partner_name}!", ephemeral=True)

    except Exception as e:
         logger.error(f"Erro ao verificar perfis para /casar (MongoDB): {e}", exc_info=True)
         return await interaction.response.send_message("Ocorreu um erro ao verificar os perfis. Tente novamente.", ephemeral=True)

    if client.pending_proposals.get(target.id):
        return await interaction.response.send_message(f"{target.display_name} já tem uma proposta de casamento pendente.", ephemeral=True)

    client.pending_proposals[target.id] = proposer.id
    view = ProposalView(proposer_id=proposer.id, target_id=target.id)

    try:
        await interaction.response.send_message(f"💍 {proposer.mention} pediu {target.mention} em casamento! {target.mention}, você tem 5 minutos para responder:", view=view)
        view.message = await interaction.original_response()
    except Exception as e:
         logger.error(f"Erro ao enviar proposta de casamento: {e}", exc_info=True)
         if target.id in client.pending_proposals and client.pending_proposals[target.id] == proposer.id:
              del client.pending_proposals[target.id]
         try:
             await interaction.followup.send("Ocorreu um erro ao enviar a proposta.", ephemeral=True)
         except:
             pass

@client.tree.command(name="divorciar", description="Termine o seu casamento.")
@app_commands.guilds(GUILD_ID_OBJ)
async def divorciar(interaction: discord.Interaction): # <-- SEM SELF
    user = interaction.user
    user_id_str = str(user.id)

    try:
        # Pega o perfil do usuário
        user_profile = await get_profile_bot(user_id_str)

        partner_id = user_profile.get("casado_com_id")
        if partner_id is None:
             await interaction.response.send_message("Você não está casado(a).", ephemeral=True)
             return

        partner_id_str = str(partner_id)

        # 1. Remove o casamento do usuário
        await client.profiles.update_one(
            {"_id": user_id_str},
            {"$set": {"casado_com_id": None, "casado_com_nome": None}}
        )

        # 2. Remove o casamento do parceiro (se ele ainda existir e estiver casado com o usuário)
        await client.profiles.update_one(
            {"_id": partner_id_str, "casado_com_id": user.id}, # Apenas se o parceiro ainda estiver casado com o usuário
            {"$set": {"casado_com_id": None, "casado_com_nome": None}}
        )

        await interaction.response.send_message("Você se divorciou.", ephemeral=True)

    except Exception as e:
        logger.error(f"Erro ao divorciar {user_id_str} (MongoDB): {e}", exc_info=True)
        await interaction.response.send_message("Ocorreu um erro ao processar o divórcio.", ephemeral=True)

# --- Comandos (Spotify) ---
@client.tree.command(name="spotify", description="Mostra sua música atual no Spotify e controles.")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.checks.cooldown(1, 5.0, key=lambda i: i.user.id)
async def spotify_cmd(interaction: discord.Interaction): # <-- SEM SELF
    await interaction.response.defer(ephemeral=True)
    discord_id = str(interaction.user.id)

    if not await get_tokens_for_user(discord_id):
        view = LoginView(make_spotify_oauth_url(discord_id))
        return await interaction.followup.send("Conecte seu Spotify.", view=view, ephemeral=True)

    try:
        player_state = await spotify_request(client.http_session, discord_id, "GET", "/me/player?market=from_token")
        if not player_state or not player_state.get("item"):
            return await interaction.edit_original_response(content="Nada tocando no Spotify no momento.", embed=None, view=None, attachments=[])

        image_buffer = await create_spotify_embed_image(client.http_session, player_state)
        file = discord.File(image_buffer, filename="spotify_card.png")
        embed = discord.Embed(color=0x1DB954)
        embed.set_image(url="attachment://spotify_card.png")
        device = player_state.get('device', {})
        embed.set_footer(text=f"Em {device.get('name','Spotify')} • Vol: {device.get('volume_percent','N/A')}%", icon_url="https://i.imgur.com/tG36kM6.png")
        view = PlaybackControlsView(user_id=discord_id, player_state=player_state, main_interaction=interaction)
        await interaction.edit_original_response(content=None, embed=embed, view=view, attachments=[file])

    except (SpotifyAuthError, SpotifyAPIError) as e:
        await interaction.edit_original_response(content=f"❌ Erro Spotify: {e}", view=None, embed=None, attachments=[])
    except FileNotFoundError as e:
        logger.error(f"/spotify: {e}", exc_info=True)
        await interaction.edit_original_response(content=f"Erro: Arquivo '{e.filename}' não encontrado.", view=None, embed=None, attachments=[])
    except Exception as e:
        logger.exception(f"/spotify erro para {discord_id}:")
        await interaction.edit_original_response(content=f"Erro inesperado ao buscar Spotify: {type(e).__name__}", view=None, embed=None, attachments=[])

@spotify_cmd.error
async def spotify_cmd_error(interaction: discord.Interaction, error: app_commands.AppCommandError):
    if isinstance(error, app_commands.CommandOnCooldown):
        await interaction.response.send_message(f"Aguarde {error.retry_after:.1f}s.", ephemeral=True)
    else:
        logger.error(f"Erro /spotify: {error}", exc_info=True)
        if not interaction.response.is_done():
            await interaction.response.send_message("Erro.", ephemeral=True)

@client.tree.command(name="perfilspotify", description="Mostra resumo do seu perfil Spotify.")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.checks.cooldown(1, 10.0, key=lambda i: i.user.id)
async def perfilspotify_cmd(interaction: discord.Interaction): # <-- SEM SELF
    await interaction.response.defer(ephemeral=False)
    discord_id = str(interaction.user.id)

    if not await get_tokens_for_user(discord_id):
        return await interaction.followup.send("Conecte seu Spotify.", view=LoginView(make_spotify_oauth_url(discord_id)), ephemeral=True)

    try:
        results = await asyncio.gather(
            spotify_request(client.http_session, discord_id, "GET", "/me"),
            spotify_request(client.http_session, discord_id, "GET", "/me/playlists?limit=50"),
            spotify_request(client.http_session, discord_id, "GET", "/me/top/tracks?time_range=short_term&limit=3"),
            spotify_request(client.http_session, discord_id, "GET", "/me/top/artists?time_range=short_term&limit=3"),
            spotify_request(client.http_session, discord_id, "GET", "/me/player/currently-playing"),
            return_exceptions=True
        )
        profile, playlists, tracks, artists, playing = results

        if any(isinstance(r, SpotifyAPIError) and "permissões" in str(r) for r in results):
            return await interaction.followup.send("Preciso de permissão para Tops/Playlists. Faça login novamente.", view=LoginView(make_spotify_oauth_url(discord_id)), ephemeral=True)

        if not isinstance(profile, dict):
            error_msg = f"Erro ao buscar perfil Spotify: {profile}" if isinstance(profile, Exception) else "Não foi possível buscar perfil."
            return await interaction.followup.send(error_msg, ephemeral=True)

        embed = discord.Embed(color=0x1DB954, title=f"Perfil de {profile.get('display_name', 'Usuário')}")
        if images := profile.get("images", []):
            embed.set_thumbnail(url=images[0]["url"])

        embed.add_field(name="Seguidores", value=f"`{profile.get('followers', {}).get('total', 0)}`")
        embed.add_field(name="País", value=f"`{profile.get('country', 'N/A')}`")
        embed.add_field(name="Playlists", value=f"`{playlists.get('total', 0) if isinstance(playlists, dict) else 'N/A'}`")

        if isinstance(playlists, dict) and (items := playlists.get("items")):
            plist_txt = "\n".join([f"• [{p['name'][:30]}]({p['external_urls']['spotify']}) ({p['tracks']['total']} faixas)" for p in items[:5]])
            embed.add_field(name="Playlists Destaque", value=plist_txt + (f"\n*... e mais {playlists['total']-5}*" if playlists['total']>5 else ""), inline=False)
        elif isinstance(playlists, Exception):
            embed.add_field(name="Playlists Destaque", value="Erro ao buscar.", inline=False)

        medals=["🥇","🥈","🥉"]
        if isinstance(tracks, dict) and tracks.get('items'):
            track_list="\n".join([f"{medals[i]} [{t['name'][:30]}]({t['external_urls']['spotify']})" for i,t in enumerate(tracks['items'])])
        else:
            track_list = "Erro ao buscar." if isinstance(tracks, Exception) else "N/A"
        embed.add_field(name="Top Músicas Mês", value=track_list, inline=True)

        if isinstance(artists, dict) and artists.get('items'):
            artist_list="\n".join([f"{medals[i]} [{a['name'][:30]}]({a['external_urls']['spotify']})" for i,a in enumerate(artists['items'])])
        else:
            artist_list = "Erro ao buscar." if isinstance(artists, Exception) else "N/A"
        embed.add_field(name="Top Artistas Mês", value=artist_list, inline=True)

        if isinstance(playing, dict) and playing.get("item"):
            embed.set_footer(text=f"Ouvindo {playing['item'].get('name', 'música')} agora", icon_url="https://i.imgur.com/tG36kM6.png")

        view = View()
        view.add_item(Button(label="Abrir Perfil Spotify", url=profile.get("external_urls",{}).get("spotify","https://spotify.com"), emoji="🎵"))
        await interaction.followup.send(embed=embed, view=view)

    except Exception as e:
        logger.exception("/perfilspotify erro:")
        await interaction.followup.send(f"Erro inesperado: {e}", ephemeral=True)

@perfilspotify_cmd.error
async def perfilspotify_cmd_error(interaction: discord.Interaction, error: app_commands.AppCommandError):
    if isinstance(error, app_commands.CommandOnCooldown):
        await interaction.response.send_message(f"Aguarde {error.retry_after:.1f}s.", ephemeral=True)
    else:
        logger.error(f"Erro /perfilspotify: {error}", exc_info=True)
        if not interaction.response.is_done():
            await interaction.response.send_message("Erro.", ephemeral=True)

@client.tree.command(name='ping_status', description='Verifique o status e desempenho do bot')
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.checks.cooldown(1, 5.0, key=lambda i: i.user.id)
async def ping_status(interaction: discord.Interaction): # <-- SEM SELF
    msg = None
    try:
        await interaction.response.defer(ephemeral=True)

        start_time = time.perf_counter()
        msg = await interaction.followup.send("Calculando latências...", ephemeral=True, wait=True)
        end_time = time.perf_counter()

        api_latency = round((end_time - start_time) * 1000)
        ws_latency = round(client.latency * 1000)

        # Google API com Timeout
        google_status = "N/A"
        google_latency = None
        try:
            google_start = time.perf_counter()
            async with client.http_session.get("https://generativelanguage.googleapis.com/v1beta/models", timeout=3.0) as r:
                google_end = time.perf_counter()
                google_latency = round((google_end - google_start) * 1000)
                google_status = f"Online ({google_latency}ms)" if r.status == 200 else f"Status {r.status}"
        except asyncio.TimeoutError:
            google_status = "Timeout"
            logger.warning("Google API check timed out in /ping_status")
        except aiohttp.ClientError as google_e:
            google_status = f"Erro Conexão"
            logger.warning(f"Google API connection error in /ping_status: {google_e}")
        except Exception as google_e:
             google_status = f"Erro Inesperado ({type(google_e).__name__})"
             logger.error(f"Unexpected error checking Google API in /ping_status: {google_e}", exc_info=False)

        # Sistema
        sys_info = {
            "OS": f"{platform.system()} {platform.release()}",
            "Arch": platform.machine(),
            "Py": platform.python_version(),
            "DPy": discord.__version__
        }

        # Stats
        shard = f"{(interaction.guild.shard_id if interaction.guild else 0)+1}/{client.shard_count or 1}"
        stats = {
            "Guildas": len(client.guilds),
            "Usuários": sum(g.member_count for g in client.guilds if g.member_count is not None),
            "Shard": shard,
            "Ver": client.version,
            "Dev": "POZEz" # Substitua se necessário
        }

        # Embed
        color = 0x00ff00 if api_latency < 300 and ws_latency < 300 else 0xffa500 if api_latency < 1000 or ws_latency < 1000 else 0xff0000
        embed = discord.Embed(
            title="📊 Status Bot",
            color=color,
            timestamp=interaction.created_at
        )
        embed.add_field(name="💻 Desempenho", value=f"• API Discord: `{api_latency}ms`\n• API Google: `{google_status}`\n• WebSocket: `{ws_latency}ms`", inline=False)
        embed.add_field(name="📦 Sistema", value=f"• OS: `{sys_info['OS']}`\n• Arch: `{sys_info['Arch']}`\n• Python: `{sys_info['Py']}`\n• Discord.py: `{sys_info['DPy']}`", inline=False)
        embed.add_field(name="📈 Estatísticas", value=f"• Guildas: `{stats['Guildas']}`\n• Usuários: `{stats['Usuários']}`\n• Shard: `{stats['Shard']}`\n• Versão: `{stats['Ver']}`\n• Dev: `{stats['Dev']}`", inline=False)
        embed.set_footer(text=f"Req por {interaction.user.display_name}", icon_url=interaction.user.display_avatar.url if interaction.user.display_avatar else None)
        embed.set_thumbnail(url=client.user.display_avatar.url if client.user.display_avatar else None)

        await msg.edit(content=None, embed=embed)

    except Exception as e:
        logger.error(f"Erro principal no /ping_status: {e}", exc_info=True)
        if msg:
            try:
                await msg.edit(content=f"❌ Erro ao obter status: {type(e).__name__}. Verifique os logs.")
            except (discord.HTTPException, discord.NotFound) as edit_e:
                 logger.error(f"Falha ao editar mensagem de erro no /ping_status: {edit_e}")
        else:
             try:
                 await interaction.followup.send(f"❌ Erro ao obter status: {type(e).__name__}. Verifique os logs.", ephemeral=True)
             except (discord.HTTPException, discord.NotFound) as follow_e:
                  logger.error(f"Falha ao enviar followup de erro no /ping_status: {follow_e}")

@ping_status.error
async def ping_status_error(interaction: discord.Interaction, error: app_commands.AppCommandError):
     if isinstance(error, app_commands.CommandOnCooldown):
         await interaction.response.send_message(f"Comando em cooldown! Tente novamente em {error.retry_after:.1f} segundos.", ephemeral=True)
     elif isinstance(error, app_commands.CommandInvokeError):
         # O erro original (e) já foi tratado e logado dentro do comando
         pass
     else:
         logger.error(f"Erro não tratado no /ping_status handler: {error}", exc_info=True)
         if not interaction.response.is_done():
             try:
                 await interaction.response.send_message("Ocorreu um erro inesperado.", ephemeral=True)
             except discord.HTTPException: pass

@client.tree.command(name='addemoji', description='Adiciona um emoji ao servidor.')
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.describe(name='Nome do emoji', url='URL do emoji (ou envie anexo)')
@app_commands.checks.has_permissions(manage_emojis=True)
async def addemoji(interaction: discord.Interaction, name: str, url: str = None, attachment: discord.Attachment = None): # <-- SEM SELF
    if ' ' in name:
        return await interaction.response.send_message('Nome não pode ter espaços.', ephemeral=True)
    emoji_data = None
    source = ""
    await interaction.response.defer(ephemeral=True)

    if url:
        source = "URL"
        try:
            emoji_data = await fetch_image(client.http_session, url)
            if not emoji_data:
                 return await interaction.followup.send(f"Falha download URL (Status não foi 200 ou imagem vazia).", ephemeral=True)
            if len(emoji_data) > 256 * 1024:
                return await interaction.followup.send("Emoji muito grande (> 256KB).", ephemeral=True)
        except Exception as e:
            return await interaction.followup.send(f"Erro ao processar URL: {e}", ephemeral=True)
    elif attachment:
        source = "Anexo"
        if attachment.size > 256 * 1024:
            return await interaction.followup.send("Emoji muito grande (> 256KB).", ephemeral=True)
        emoji_data = await attachment.read()
    else:
        return await interaction.followup.send('Precisa de URL ou anexo.', ephemeral=True)

    if emoji_data:
        try:
             emoji = await interaction.guild.create_custom_emoji(name=name, image=emoji_data, reason=f"Adicionado por {interaction.user} via /addemoji ({source})")
             await interaction.followup.send(f'Emoji {emoji} adicionado!', ephemeral=False) # Envia público
        except discord.Forbidden:
            await interaction.followup.send("Sem permissão para criar emojis.", ephemeral=True)
        except discord.HTTPException as e:
            await interaction.followup.send(f'Falha API Discord: {e}', ephemeral=True)
        except ValueError as e:
            await interaction.followup.send(f'Erro: {e}', ephemeral=True)

@addemoji.error
async def addemoji_error(i:discord.Interaction, e:app_commands.AppCommandError):
    if isinstance(e, app_commands.MissingPermissions):
        await i.response.send_message("Sem permissão 'Gerenciar Emojis'.", ephemeral=True)
    else:
        logger.error(f"Erro /addemoji: {e}", exc_info=True)
        if not i.response.is_done():
             await i.response.send_message(f"Erro: {e}", ephemeral=True)

@client.tree.command(name='clear', description='Limpa mensagens do canal (até 100).')
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.describe(amount='Quantidade de mensagens a excluir (1-100)')
@app_commands.checks.has_permissions(manage_messages=True)
async def clear(interaction: discord.Interaction, amount: app_commands.Range[int, 1, 100]): # <-- SEM SELF
    if not interaction.app_permissions.manage_messages:
        return await interaction.response.send_message("Eu não tenho permissão para 'Gerenciar Mensagens' neste canal.", ephemeral=True)

    await interaction.response.defer(ephemeral=True, thinking=True)

    deleted_count = 0
    failed_count = 0
    fourteen_days_ago = discord.utils.utcnow() - timedelta(days=14)
    messages_to_delete_bulk = []
    messages_to_delete_individually = []

    try:
        fetched_messages = [msg async for msg in interaction.channel.history(limit=amount)]
        if not fetched_messages:
            await interaction.followup.send("Nenhuma mensagem encontrada para excluir.", ephemeral=True)
            return

        for msg in fetched_messages:
            if msg.created_at > fourteen_days_ago:
                messages_to_delete_bulk.append(msg)
            else:
                messages_to_delete_individually.append(msg)

        if messages_to_delete_bulk:
            try:
                # Usa purge (que é bulk)
                deleted_bulk = await interaction.channel.purge(
                    limit=len(messages_to_delete_bulk),
                    check=lambda m: m.id in [msg.id for msg in messages_to_delete_bulk],
                    bulk=True
                )
                deleted_count += len(deleted_bulk)
                logger.info(f"Bulk deleted {len(deleted_bulk)} messages newer than 14 days.")
            except discord.HTTPException as e:
                logger.warning(f"Error during bulk delete (might be partial, trying individual): {e}")
                messages_to_delete_individually.extend(messages_to_delete_bulk) # Adiciona falhas na lista individual
            except discord.Forbidden:
                 logger.warning(f"No permission for bulk delete in {interaction.channel.name}.")
                 failed_count += len(messages_to_delete_bulk)

        if messages_to_delete_individually:
            logger.info(f"Attempting to individually delete {len(messages_to_delete_individually)} messages.")
            for msg in messages_to_delete_individually:
                # Pula se já foi deletado no bulk (caso purge tenha pego alguns)
                if 'deleted_bulk' in locals() and msg.id in [d.id for d in deleted_bulk]:
                    continue
                try:
                    await msg.delete()
                    deleted_count += 1
                    await asyncio.sleep(0.6)
                except discord.Forbidden:
                     failed_count += 1
                     logger.warning(f"No permission to delete message {msg.id} (individual). Skipping remaining.")
                     break
                except discord.NotFound:
                     logger.warning(f"Message {msg.id} not found during individual delete.")
                except discord.HTTPException as e:
                     failed_count += 1
                     logger.error(f"HTTP Error deleting message {msg.id} (individual): {e}")
                     await asyncio.sleep(1)

        result_message = f"✅ {deleted_count} mensagens excluídas."
        if failed_count > 0:
            result_message += f"\n⚠️ Falha ao excluir {failed_count} mensagens."

        try:
             await interaction.followup.send(result_message, ephemeral=True)
        except discord.NotFound:
             logger.warning("Interaction expired before /clear could send followup.")
             return

    except discord.Forbidden:
        logger.warning(f"No permission to read history or manage messages in {interaction.channel.name}.")
        try:
             await interaction.followup.send('Não tenho permissão para ler ou apagar mensagens neste canal.', ephemeral=True)
        except discord.NotFound:
             pass
    except Exception as e:
        logger.error(f"Erro inesperado no /clear: {e}", exc_info=True)
        try:
            await interaction.followup.send(f'Ocorreu um erro inesperado: {e}', ephemeral=True)
        except discord.NotFound:
             pass

@clear.error
async def clear_error(i:discord.Interaction, e:app_commands.AppCommandError):
    if isinstance(e, app_commands.MissingPermissions):
        await i.response.send_message("Você não tem permissão para 'Gerenciar Mensagens'.", ephemeral=True)
    elif isinstance(e, app_commands.RangeError):
        await i.response.send_message("A quantidade deve ser entre 1 e 100.", ephemeral=True)
    elif isinstance(e, app_commands.CommandInvokeError):
        pass # O erro já foi tratado dentro do comando
    else:
        logger.error(f"Erro não tratado no /clear handler: {e}", exc_info=True)
        if not i.response.is_done():
             await i.response.send_message(f"Erro: {e}", ephemeral=True)

@client.tree.command(name='steal', description='Copia emoji de outro server.')
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.describe(emoji='O emoji a ser copiado (<:nome:id>)', name='Novo nome (opcional)')
@app_commands.checks.has_permissions(manage_emojis=True)
async def steal(interaction: discord.Interaction, emoji: str, name: str = None): # <-- SEM SELF
    e_id, e_name, is_anim = extract_emoji_info(emoji)
    if not e_id:
        return await interaction.response.send_message(
            "Formato inválido. Use `<:nome:id>` ou `<a:nome:id>`.", ephemeral=True
        )

    ext = 'gif' if is_anim else 'png'
    url = f"https://cdn.discordapp.com/emojis/{e_id}.{ext}?quality=lossless"

    await interaction.response.defer(ephemeral=True)

    e_data = await fetch_image(client.http_session, url)
    if not e_data:
        return await interaction.followup.send("Emoji não encontrado ou URL inválida.", ephemeral=True)

    if len(e_data) > 256 * 1024:
        return await interaction.followup.send("Emoji original muito grande (> 256KB).", ephemeral=True)

    name = name or e_name
    try:
        new_e = await interaction.guild.create_custom_emoji(
            name=name,
            image=e_data,
            reason=f"Copiado por {interaction.user} via /steal"
        )
        await interaction.followup.send(f'Emoji {new_e} copiado com sucesso!', ephemeral=False)
    except discord.Forbidden:
        await interaction.followup.send("Sem permissão para criar emojis.", ephemeral=True)
    except discord.HTTPException as exc:
        await interaction.followup.send(f'Falha na API do Discord: {exc}', ephemeral=True)
    except ValueError as exc:
        await interaction.followup.send(f'Erro: {exc}', ephemeral=True)

@steal.error
async def steal_error(i: discord.Interaction, e: app_commands.AppCommandError):
    if isinstance(e, app_commands.MissingPermissions):
        await i.response.send_message("Sem permissão 'Gerenciar Emojis'.", ephemeral=True)
    else:
        logger.error(f"Erro /steal: {e}", exc_info=True)
        if not i.response.is_done():
            await i.response.send_message(f"Erro: {e}", ephemeral=True)

@client.tree.command(name="userinfo", description="Mostra informações sobre um usuário")
@app_commands.guilds(GUILD_ID_OBJ)
async def userinfo(interaction: discord.Interaction, membro: discord.Member = None): # <-- SEM SELF
    user = membro or interaction.user
    e_base = discord.Embed(title=f"Informações de {user.display_name}", color=user.color or discord.Color.blurple())
    e_base.set_thumbnail(url=user.display_avatar.url if user.display_avatar else None)
    e_base.add_field(name="Nome", value=f"`{user.name}`", inline=True)
    e_base.add_field(name="Tag", value=(f"`#{user.discriminator}`" if getattr(user, "discriminator", "0") != '0' else "`#N/A`"), inline=True)
    e_base.add_field(name="ID", value=f"`{user.id}`", inline=False)
    e_base.add_field(name="📅 Conta criada em", value=f"<t:{int(user.created_at.timestamp())}:F>", inline=False)

    e_serv = discord.Embed(title="Informações no Servidor", color=user.color or discord.Color.blurple())
    view = None
    if isinstance(user, discord.Member):
        roles = [r.mention for r in user.roles if not r.is_default()]
        r_list = list(roles)
        e_serv.add_field(name="Apelido", value=f"`{user.nick or 'Nenhum'}`", inline=True)
        e_serv.add_field(name="📅 Entrou em", value=f"<t:{int(user.joined_at.timestamp())}:F>", inline=True)
        e_serv.add_field(name="Maior cargo", value=(user.top_role.mention if user.top_role else "Nenhum"), inline=True)
        e_serv.add_field(name="Nº Cargos", value=f"`{len(r_list)}`", inline=True)
        view = RolesView(r_list, user)

        try:
            flags = [f.name for f in user.public_flags.all()]
            if flags:
                flags_str = ", ".join(f.replace("_", " ").title() for f in flags)
                e_serv.add_field(name="🚩 Flags", value=flags_str, inline=False)
        except Exception:
            pass
    else:
        e_serv.description = "Este usuário não é membro deste servidor."

    await interaction.response.send_message(embeds=[e_base, e_serv], view=view)
    if view:
        view.message = await interaction.original_response()

@client.tree.command(name='serverinfo', description='Mostra informações do servidor.')
@app_commands.guilds(GUILD_ID_OBJ)
async def serverinfo(interaction: discord.Interaction): # <-- SEM SELF
    g = interaction.guild
    if g is None:
        return await interaction.response.send_message("Comando só pode ser usado em um servidor.", ephemeral=True)

    owner = g.owner # Pode ser None se o bot não tiver acesso
    owner_color = (owner.color if owner else discord.Color.default())
    e1 = discord.Embed(title=f"{g.name}", color=owner_color)
    e1.set_thumbnail(url=g.icon.url if g.icon else discord.Embed.Empty)
    e1.add_field(name="ID", value=f"`{g.id}`", inline=True)
    e1.add_field(name="Dono", value=owner.mention if owner else "N/A", inline=True)
    e1.add_field(name="Membros", value=f"{g.member_count}", inline=True)
    e1.add_field(name="Boosts", value=f"{g.premium_subscription_count} (Nível {g.premium_tier})", inline=True)

    cargos = [c for c in g.roles if not c.is_default()]
    canais = {
        "Texto": len(g.text_channels),
        "Voz": len(g.voice_channels),
        "Palco": len(g.stage_channels),
        "Categorias": len(g.categories),
    }
    e2 = discord.Embed(color=owner_color)
    e2.add_field(name="Canais", value="\n".join([f"• **{t}:** {q}" for t, q in canais.items()]), inline=True)
    e2.add_field(name="Cargos", value=f"{len(cargos)}", inline=True)
    if g.banner:
        e2.set_image(url=g.banner.url)

    view = discord.ui.View(timeout=300)
    view.add_item(InfoExtraBotao())

    await interaction.response.send_message(embeds=[e1, e2], view=view)
    message = await interaction.original_response()

    # Passa a mensagem para a view para que o on_timeout possa editá-la
    view.message = message

    # O on_timeout da view lidará com a desativação dos botões

# ---------------------- Comandos de Moderação ----------------------
@client.tree.command(name='ban', description='Bane um usuário.')
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.describe(user='Usuário a ser banido', reason='Motivo do banimento')
@app_commands.checks.has_permissions(ban_members=True)
async def ban(interaction: discord.Interaction, user: discord.Member, reason: str = 'Nenhuma razão fornecida'): # <-- SEM SELF
    if user == interaction.user:
        return await interaction.response.send_message("Você não pode se banir.", ephemeral=True)

    me = interaction.guild.me
    if me and user.top_role >= me.top_role:
        return await interaction.response.send_message("Não posso banir (meu cargo é mais baixo).", ephemeral=True)
    if (user.top_role >= interaction.user.top_role) and (interaction.guild.owner_id != interaction.user.id):
        return await interaction.response.send_message("Não pode banir (cargo do usuário é maior/igual ao seu).", ephemeral=True)

    try:
        await user.ban(reason=f"{reason} (Banido por {interaction.user.display_name})")
        await interaction.response.send_message(f'{user.mention} foi banido. Motivo: {reason}', ephemeral=True)
        await log_action(interaction.guild, f"BAN: {user} ({user.id}) banido por {interaction.user}. Motivo: {reason}")
    except discord.Forbidden:
        await interaction.response.send_message(f'Sem permissão Discord para banir {user.mention}.', ephemeral=True)
    except Exception as exc:
        logger.exception("Erro /ban")
        await interaction.response.send_message(f'Erro inesperado: {exc}', ephemeral=True)

@ban.error
async def ban_error(i: discord.Interaction, e: app_commands.AppCommandError):
    if isinstance(e, app_commands.MissingPermissions):
        await i.response.send_message("Sem permissão 'Banir Membros'.", ephemeral=True)
    else:
        logger.error(f"Erro /ban: {e}", exc_info=True)
        if not i.response.is_done():
            try:
                await i.response.send_message(f"Erro: {e}", ephemeral=True)
            except Exception:
                pass

@client.tree.command(name='trancar', description='Tranca o canal atual (impede @everyone de enviar msgs).')
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.checks.has_permissions(manage_channels=True)
async def trancar(interaction: discord.Interaction): # <-- SEM SELF
    channel = interaction.channel
    if not isinstance(channel, discord.TextChannel):
        return await interaction.response.send_message("Só funciona em canais de texto.", ephemeral=True)
    try:
        current_perms = channel.overwrites_for(interaction.guild.default_role)
        current_perms.send_messages = False
        await channel.set_permissions(interaction.guild.default_role, overwrite=current_perms, reason=f"Canal trancado por {interaction.user.display_name}")
        await interaction.response.send_message(f'🔒 Canal {channel.mention} foi trancado.', ephemeral=True)
        await log_action(interaction.guild, f"LOCK: Canal {channel.name} ({channel.id}) trancado por {interaction.user}.")
    except discord.Forbidden:
        await interaction.response.send_message('Sem permissão Discord para gerenciar permissões.', ephemeral=True)
    except Exception as exc:
        logger.exception("Erro /trancar")
        await interaction.response.send_message(f'Erro: {exc}', ephemeral=True)

@trancar.error
async def trancar_error(i: discord.Interaction, e: app_commands.AppCommandError):
    if isinstance(e, app_commands.MissingPermissions):
        await i.response.send_message("Sem permissão 'Gerenciar Canais'.", ephemeral=True)
    else:
        if not i.response.is_done():
            await i.response.send_message(f"Erro: {e}", ephemeral=True)

@client.tree.command(name='destrancar', description='Destranca o canal atual (permite @everyone enviar msgs).')
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.checks.has_permissions(manage_channels=True)
async def destrancar(interaction: discord.Interaction): # <-- SEM SELF
    channel = interaction.channel
    if not isinstance(channel, discord.TextChannel):
        return await interaction.response.send_message("Só funciona em canais de texto.", ephemeral=True)
    try:
        current_perms = channel.overwrites_for(interaction.guild.default_role)
        current_perms.send_messages = None  # volta a herdar
        await channel.set_permissions(interaction.guild.default_role, overwrite=current_perms, reason=f"Canal destrancado por {interaction.user.display_name}")
        await interaction.response.send_message(f'🔓 Canal {channel.mention} foi destrancado.', ephemeral=True)
        await log_action(interaction.guild, f"UNLOCK: Canal {channel.name} ({channel.id}) destrancado por {interaction.user}.")
    except discord.Forbidden:
        await interaction.response.send_message('Sem permissão Discord para gerenciar permissões.', ephemeral=True)
    except Exception as exc:
        logger.exception("Erro /destrancar")
        await interaction.response.send_message(f'Erro: {exc}', ephemeral=True)

@destrancar.error
async def destrancar_error(i: discord.Interaction, e: app_commands.AppCommandError):
    if isinstance(e, app_commands.MissingPermissions):
        await i.response.send_message("Sem permissão 'Gerenciar Canais'.", ephemeral=True)
    else:
         if not i.response.is_done():
            await i.response.send_message(f"Erro: {e}", ephemeral=True)

@client.tree.command(name="painel", description="Painel moderação simplificado (admin).")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.checks.has_permissions(administrator=True)
async def moderar(interaction: discord.Interaction, membro: discord.Member): # <-- SEM SELF
    if membro.id == OWNER_ID:
        return await interaction.response.send_message("Não pode usar em si mesmo.", ephemeral=True)
    if membro.id == client.user.id:
        return await interaction.response.send_message("Não pode usar em mim.", ephemeral=True)
    if interaction.guild and interaction.guild.me and membro.top_role >= interaction.guild.me.top_role:
        return await interaction.response.send_message("Não posso gerenciar (cargo mais alto).", ephemeral=True)

    count = "N/A"
    try:
        after = discord.utils.utcnow() - timedelta(days=7)
        count = 0
        async for msg in interaction.channel.history(limit=1000, after=after):
            if msg.author.id == membro.id:
                count += 1
    except Exception:
        count = "N/A" # Ignora se não puder ler o histórico

    embed = discord.Embed(title=f"🔰 Painel Mod - {membro.display_name}", color=discord.Color.red())
    embed.add_field(name="ID", value=f"`{membro.id}`", inline=False)

    if membro.created_at:
        embed.add_field(name="Conta criada", value=f"<t:{int(membro.created_at.timestamp())}:R>", inline=False)
    if membro.joined_at:
        embed.add_field(name="Entrou em", value=f"<t:{int(membro.joined_at.timestamp())}:R>", inline=False)

    embed.add_field(name="Status", value=str(membro.status).title(), inline=False)

    cargos = ", ".join([r.mention for r in membro.roles if not r.is_default()]) if len(membro.roles) > 1 else "Nenhum"
    cargos = cargos[:1020] + "..." if len(cargos) > 1024 else cargos
    embed.add_field(name=f"Cargos ({len(membro.roles) - 1})", value=cargos or "Nenhum", inline=False)

    embed.add_field(name="Cargo + alto", value=f"{membro.top_role.mention} (pos {membro.top_role.position})", inline=False)
    embed.add_field(name="Msgs (7d)", value=str(count), inline=True)
    embed.add_field(name="Bot", value="Sim" if membro.bot else "Não", inline=True)
    embed.add_field(name="Admin Server", value="Sim" if membro.guild_permissions.administrator else "Não", inline=True)

    if membro.voice and membro.voice.channel:
        embed.add_field(name="Canal voz",
                        value=f"{membro.voice.channel.mention} ({'Server Mudo' if membro.voice.mute else 'Não Mudo'})",
                        inline=False)

    if membro.display_avatar:
        embed.set_thumbnail(url=membro.display_avatar.url)

    view = ModerationView(membro, interaction)
    await interaction.response.send_message(embed=embed, view=view, ephemeral=True)

    try:
        view.message = await interaction.original_response()
    except Exception:
        view.message = None

@moderar.error
async def mod_error(interaction: discord.Interaction, error: app_commands.AppCommandError):
    if isinstance(error, app_commands.MissingPermissions):
        await interaction.response.send_message("Só admins do servidor.", ephemeral=True)
    else:
        logger.error(f"Erro /painel: {error}", exc_info=True)
        if not interaction.response.is_done():
            await interaction.response.send_message(f"Erro /painel: {error}", ephemeral=True)

@client.tree.command(name="topconvites", description="Top 10 convites do servidor.")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.checks.has_permissions(manage_guild=True)
async def topconvites(interaction: discord.Interaction): # <-- SEM SELF
    try:
        invites = await interaction.guild.invites()
        sorted_invites = sorted([inv for inv in invites if getattr(inv, 'uses', None) is not None],
                                key=lambda inv: inv.uses or 0,
                                reverse=True)[:10]
    except discord.Forbidden:
        return await interaction.response.send_message("Sem permissão Discord para ver convites.", ephemeral=True)
    except Exception as e:
        logger.error(f"Erro /topconvites: {e}")
        return await interaction.response.send_message(f"Erro: {e}", ephemeral=True)

    if sorted_invites:
        resp = "\n".join([
            f"• `{inv.code}`: **{inv.uses or 0}** usos ({inv.inviter.mention if inv.inviter else 'N/A'})"
            for inv in sorted_invites
        ])
        embed = discord.Embed(title="🏆 Top 10 Convites", description=resp, color=discord.Color.blue())
    else:
        embed = discord.Embed(title="🏆 Top 10 Convites", description="Nenhum convite com usos encontrado.", color=discord.Color.blue())

    await interaction.response.send_message(embed=embed)

@topconvites.error
async def topconvites_error(interaction: discord.Interaction, error: app_commands.AppCommandError):
    if isinstance(error, app_commands.MissingPermissions):
        await interaction.response.send_message("Sem permissão 'Gerenciar Servidor'.", ephemeral=True)
    else:
        if not interaction.response.is_done():
            await interaction.response.send_message(f"Erro: {error}", ephemeral=True)

@client.tree.command(name="conviteinfo", description="Info detalhada de um convite.")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.describe(code="Código do convite.")
@app_commands.checks.has_permissions(manage_guild=True)
async def conviteinfo(interaction: discord.Interaction, code: str): # <-- SEM SELF
    invite = None
    try:
        invite = await client.fetch_invite(code, with_counts=True)
    except discord.NotFound:
        return await interaction.response.send_message(f"Convite `{code}` não encontrado ou expirado.", ephemeral=True)
    except Exception as e:
        logger.error(f"Erro /conviteinfo fetch: {e}")
        return await interaction.response.send_message(f"Erro ao buscar convite: {e}", ephemeral=True)

    if not invite.guild or invite.guild.id != interaction.guild.id:
        guild_name = invite.guild.name if invite.guild and getattr(invite.guild, 'name', None) else 'Desconhecido'
        return await interaction.response.send_message(f"Convite `{code}` é para outro servidor ({guild_name}).", ephemeral=True)

    creator = invite.inviter.mention if invite.inviter else "N/A"
    created_at = f"<t:{int(invite.created_at.timestamp())}:f>" if invite.created_at else "N/A"
    uses = invite.uses or 0
    max_uses = invite.max_uses if invite.max_uses not in (None, 0) else "∞"
    members = invite.approximate_member_count or "N/A"
    online = invite.approximate_presence_count or "N/A"

    embed = discord.Embed(title=f"ℹ️ Info Convite: `{code}`", description=f"Servidor: {invite.guild.name}", color=discord.Color.blue())
    embed.add_field(name="Criador", value=creator)
    embed.add_field(name="Criado em", value=created_at)
    embed.add_field(name="Usos", value=f"{uses}/{max_uses}")
    embed.add_field(name="Canal", value=invite.channel.mention if invite.channel else "N/A")
    embed.add_field(name="Membros", value=f"🟢{online} / {members}", inline=True)
    embed.add_field(name="Link", value=f"[discord.gg/{code}](https://discord.gg/{code})", inline=False)

    if invite.guild.icon:
        try:
            embed.set_thumbnail(url=invite.guild.icon.url)
        except Exception:
            pass

    if interaction.user.display_avatar:
        embed.set_footer(text=f"Req por {interaction.user.display_name}", icon_url=interaction.user.display_avatar.url)

    await interaction.response.send_message(embed=embed)

@conviteinfo.error
async def conviteinfo_error(interaction: discord.Interaction, error: app_commands.AppCommandError):
    if isinstance(error, app_commands.MissingPermissions):
        await interaction.response.send_message("Sem permissão 'Gerenciar Servidor'.", ephemeral=True)
    else:
        if not interaction.response.is_done():
            await interaction.response.send_message(f"Erro: {error}", ephemeral=True)

@client.tree.command(name="roleadd", description="Atribui cargo a usuário.")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.describe(usuario="Usuário a receber cargo", cargo="Cargo a ser dado")
@app_commands.checks.has_permissions(manage_roles=True)
async def roleadd(interaction: discord.Interaction, usuario: discord.Member, cargo: discord.Role): # <-- SEM SELF
    if cargo in usuario.roles:
        return await interaction.response.send_message(f"{usuario.mention} já tem {cargo.mention}.", ephemeral=True)
    if interaction.user.top_role <= cargo and interaction.guild.owner_id != interaction.user.id:
        return await interaction.response.send_message("Não pode dar cargo igual/superior ao seu.", ephemeral=True)
    if interaction.guild.me.top_role <= cargo:
        return await interaction.response.send_message("Não posso dar cargo igual/superior ao meu.", ephemeral=True)

    try:
        await usuario.add_roles(cargo, reason=f"Adicionado por {interaction.user} via /roleadd")
        await interaction.response.send_message(f"{cargo.mention} adicionado a {usuario.mention}!", ephemeral=True)
        await log_action(interaction.guild, f"ROLE ADD: {cargo.name} ({cargo.id}) adicionado a {usuario} ({usuario.id}) por {interaction.user}.")
    except discord.Forbidden:
        await interaction.response.send_message("Sem permissão Discord.", ephemeral=True)
    except discord.HTTPException as e:
        logger.error(f"Erro /roleadd HTTP: {e}")
        await interaction.response.send_message(f"Erro HTTP: {e}", ephemeral=True)

@roleadd.error
async def roleadd_error(interaction: discord.Interaction, error: app_commands.AppCommandError):
    if isinstance(error, app_commands.MissingPermissions):
        await interaction.response.send_message("Sem permissão 'Gerenciar Cargos'.", ephemeral=True)
    else:
        if not interaction.response.is_done():
            await interaction.response.send_message(f"Erro: {error}", ephemeral=True)

@client.tree.command(name="roleremove", description="Remove cargo de usuário.")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.describe(usuario="Usuário a perder cargo", cargo="Cargo a ser removido")
@app_commands.checks.has_permissions(manage_roles=True)
async def roleremove(interaction: discord.Interaction, usuario: discord.Member, cargo: discord.Role): # <-- SEM SELF
    if cargo not in usuario.roles:
        return await interaction.response.send_message(f"{usuario.mention} não tem {cargo.mention}.", ephemeral=True)
    if interaction.user.top_role <= cargo and interaction.guild.owner_id != interaction.user.id:
        return await interaction.response.send_message("Não pode remover cargo igual/superior ao seu.", ephemeral=True)
    if interaction.guild.me.top_role <= cargo:
        return await interaction.response.send_message("Não posso remover cargo igual/superior ao meu.", ephemeral=True)

    try:
        await usuario.remove_roles(cargo, reason=f"Removido por {interaction.user} via /roleremove")
        await interaction.response.send_message(f"{cargo.mention} removido de {usuario.mention}!", ephemeral=True)
        await log_action(interaction.guild, f"ROLE REMOVE: {cargo.name} ({cargo.id}) removido de {usuario} ({usuario.id}) por {interaction.user}.")
    except discord.Forbidden:
        await interaction.response.send_message("Sem permissão Discord.", ephemeral=True)
    except discord.HTTPException as e:
        logger.error(f"Erro /roleremove HTTP: {e}")
        await interaction.response.send_message(f"Erro HTTP: {e}", ephemeral=True)

@roleremove.error
async def roleremove_error(interaction: discord.Interaction, error: app_commands.AppCommandError):
    if isinstance(error, app_commands.MissingPermissions):
        await interaction.response.send_message("Sem permissão 'Gerenciar Cargos'.", ephemeral=True)
    else:
        if not interaction.response.is_done():
            await interaction.response.send_message(f"Erro: {error}", ephemeral=True)

@client.tree.command(name="petpet", description="DIVERSÃO | Faça carinho!")
@app_commands.guilds(GUILD_ID_OBJ)
async def petpet(interaction: discord.Interaction, usuario: discord.Member): # <-- SEM SELF
    if not usuario.display_avatar:
         return await interaction.response.send_message("Não consigo encontrar o avatar desse usuário.", ephemeral=True)
    url = f"https://api.popcat.xyz/pet?image={usuario.display_avatar.replace(format='png').url}"
    await interaction.response.defer()
    try:
        async with client.http_session.get(url) as r:
            if r.status == 200:
                data = await r.read()
                file = discord.File(fp=io.BytesIO(data), filename="petpet.gif")
                await interaction.followup.send(f"{interaction.user.display_name} fez carinho em {usuario.display_name}!", file=file)
            else:
                await interaction.followup.send(f"Erro API Popcat (Status: {r.status}).")
    except Exception as e:
        logger.error(f"Erro /petpet: {e}", exc_info=True)
        await interaction.followup.send("Erro inesperado ao gerar petpet.")

@client.tree.command(name="biden", description="DIVERSÃO | Crie tweet do Biden!")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.describe(texto="Texto do tweet (máx 280)")
async def biden(interaction: discord.Interaction, texto: str): # <-- SEM SELF
    if len(texto) > 280:
        return await interaction.response.send_message("Texto longo (máx 280).", ephemeral=True)

    url = f"https://api.popcat.xyz/biden?text={urllib.parse.quote(texto)}"
    await interaction.response.defer()
    try:
        async with client.http_session.get(url) as r:
            if r.status == 200:
                data = await r.read()
                file = discord.File(fp=io.BytesIO(data), filename="biden_tweet.png")
                await interaction.followup.send(f"Tweet gerado por {interaction.user.display_name}:", file=file)
            else:
                await interaction.followup.send(f"Erro API Popcat (Status: {r.status}).")
    except Exception as e:
        logger.error(f"Erro /biden: {e}", exc_info=True)
        await interaction.followup.send("Erro inesperado ao gerar tweet.")

@client.tree.command(name="ship", description="DIVERSÃO | Compatibilidade amorosa!")
@app_commands.guilds(GUILD_ID_OBJ)
async def ship(interaction: discord.Interaction, user1: discord.Member, user2: discord.Member): # <-- SEM SELF
    if user1.bot or user2.bot:
        return await interaction.response.send_message("Bots não podem amar :(", ephemeral=True)
    if user1 == user2:
        return await interaction.response.send_message("Amor próprio é importante, mas...", ephemeral=True)

    await interaction.response.defer()

    try:
        user1_profile = await get_profile_bot(user1.id)
        user2_profile = await get_profile_bot(user2.id)
        user1_married_to = user1_profile.get("casado_com_id")
        user2_married_to = user2_profile.get("casado_com_id")

        msg = None
        if user1_married_to == user2.id:
            frases = ["Ship perfeito!", "Química total!", "Almas gêmeas!", "Feitos um para o outro!"]
            msg = f"💖 **1000%** de compatibilidade! (Já são casados!)\n{random.choice(frases)}"
            return await interaction.followup.send(content=f"{user1.mention} + {user2.mention}\n{msg}")

        elif user1_married_to and user1_married_to != user2.id:
            partner1 = interaction.guild.get_member(user1_married_to)
            partner1_mention = f"<@{user1_married_to}>" if not partner1 else partner1.mention
            msg = f"💔 **0%** de compatibilidade!\n{user1.mention} já está em um relacionamento com {partner1_mention}!"
            return await interaction.followup.send(content=f"{user1.mention} + {user2.mention}\n{msg}")
        elif user2_married_to and user2_married_to != user1.id:
            partner2 = interaction.guild.get_member(user2_married_to)
            partner2_mention = f"<@{user2_married_to}>" if not partner2 else partner2.mention
            msg = f"💔 **0%** de compatibilidade!\n{user2.mention} já está em um relacionamento com {partner2_mention}!"
            return await interaction.followup.send(content=f"{user1.mention} + {user2.mention}\n{msg}")

        else:
            perc = random.randint(0, 100)
            if perc <= 25: msg_list = ["Sem chances...", "Universo disse não.", "Não combina."]
            elif perc <= 50: msg_list = ["Pode rolar, com esforço.", "Fio de esperança.", "Talvez..."]
            elif perc <= 75: msg_list = ["Clima esquentando!", "Faíscas!", "Tem potencial!"]
            else: msg_list = ["Destino!", "Casamento quando?", "Casal perfeito!"]
            msg = f"💖 **{perc}%** de compatibilidade!\n{random.choice(msg_list)}"

            def combinar_nomes(n1: str, n2: str) -> str:
                m1 = max(1, len(n1) // 2)
                m2 = len(n2) // 2
                return n1[:m1] + n2[m2:]
            ship_name = combinar_nomes(user1.display_name, user2.display_name)

            try:
                base_ship_img = Image.open("ship.png").convert("RGBA")
            except FileNotFoundError:
                return await interaction.followup.send(f"Erro: `ship.png` não encontrado.\n{user1.mention} + {user2.mention} = **{ship_name}**\n{msg}")

            avatar1_data = await fetch_image(client.http_session, user1.display_avatar.url if user1.display_avatar else None)
            avatar2_data = await fetch_image(client.http_session, user2.display_avatar.url if user2.display_avatar else None)
            if not avatar1_data or not avatar2_data:
                 await interaction.followup.send(f"Erro ao baixar avatares.\n{user1.mention} + {user2.mention} = **{ship_name}**\n{msg}")
                 return

            buffer = await asyncio.to_thread(_generate_ship_image_sync, base_ship_img, avatar1_data, avatar2_data)
            file = discord.File(fp=buffer, filename="ship_result.png")
            await interaction.followup.send(content=f"{user1.mention} + {user2.mention} = **{ship_name}**\n{msg}", file=file)

    except Exception as e:
        logger.exception("/ship erro na geração da imagem:")
        await interaction.followup.send(f"Erro ao gerar imagem: {e}")

@client.tree.command(name="gaby", description="Envia uma mensagem especial.")
@app_commands.guilds(GUILD_ID_OBJ)
async def gaby(interaction: discord.Interaction): # <-- SEM SELF
    GABY_ID = 1213820913129361519
    POZEZ_ID = 1017254600480411670

    if interaction.user.id != GABY_ID:
        await interaction.response.send_message("Desculpe, apenas a Gaby pode usar este comando! ❤️", ephemeral=True)
        return

    msg = f"❤️ {interaction.user.mention}, você é a luz da vida do <@{POZEZ_ID}>!"

    try:
        await interaction.response.send_message(msg, ephemeral=False)
    except Exception as e:
        logger.error(f"Erro ao enviar mensagem no /gaby para Gaby: {e}", exc_info=True)
        try:
             await interaction.followup.send("Ocorreu um erro ao enviar a mensagem.", ephemeral=True)
        except discord.HTTPException:
             pass

@client.tree.command(name="minecraft", description="Verifica informações de uma conta Minecraft")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.describe(identifier="Nome de usuário ou UUID do jogador")
async def minecraft(interaction: discord.Interaction, identifier: str): # <-- SEM SELF
    await interaction.response.defer(thinking=True)

    try:
        cleaned_id = identifier.replace('-', '')
        is_uuid = len(cleaned_id) == 32

        url = (
             f"https://api.mojang.com/user/profile/{cleaned_id}"
             if is_uuid
             else f"https://api.mojang.com/users/profiles/minecraft/{identifier}"
        )

        # CORRIGIDO: Usa aiohttp (async) em vez de requests (blocking)
        async with client.http_session.get(url, timeout=10) as response:
            if response.status != 200:
                return await interaction.followup.send(
                    "❌ Conta não encontrada. Verifique o nome/UUID digitado.",
                    ephemeral=True
                )
            data = await response.json()

        if not data:
             return await interaction.followup.send("❌ Conta não encontrada (resposta vazia).", ephemeral=True)

        uuid = data['id']
        username = data['name']

        # CORRIGIDO: Reutiliza a função fetch_image
        head_data = await fetch_image(client.http_session, f"https://crafatar.com/avatars/{uuid}?overlay")
        if not head_data:
            return await interaction.followup.send("❌ Falha ao carregar a cabeça do jogador", ephemeral=True)

        head_file = discord.File(io.BytesIO(head_data), filename="head.png")

        embed = discord.Embed(
            title=f"<a:MineCap:1251909415901069395> **{username}**",
            color=discord.Color.green(),
            description=f"**UUID:** `{uuid}` \n[Ver perfil no NameMc ↗](https://pt.namemc.com/profile/{uuid})"
        )
        embed.set_thumbnail(url="attachment://head.png")

        view = MinecraftView(uuid)
        await interaction.followup.send(embed=embed, file=head_file, view=view)
        view.message = await interaction.original_response()

    except (aiohttp.ClientError, asyncio.TimeoutError):
        await interaction.followup.send(
            "❌ Erro ao conectar-se aos servidores da Mojang",
            ephemeral=True
        )
    except Exception as e:
        logger.error(f"Erro /minecraft: {e}", exc_info=True)
        await interaction.followup.send(
            "⚠️ Ocorreu um erro ao processar sua solicitação.",
            ephemeral=True
        )

@client.tree.command(name="ia", description="Converse com o Gemini API")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.describe(pergunta="Sua pergunta para a IA")
async def ia(interaction: discord.Interaction, pergunta: str): # <-- SEM SELF
    try:
        await interaction.response.defer()
        response = await model.generate_content_async(pergunta)
        resp_text = response.text[:1997]+"..." if len(response.text)>2000 else response.text
        await interaction.followup.send(f"<:j_cerebrobot:1363237071011057725> **Boreal IA**\n{resp_text}")
    except Exception as e:
        logger.error(f"Erro API Gemini: {e}", exc_info=True)
        await interaction.followup.send(f"Ocorreu um erro ao contatar a IA. Tente novamente mais tarde.")

@client.tree.command(name="list_commands", description="OWNER | Lista comandos slash registrados")
@app_commands.guilds(GUILD_ID_OBJ)
async def list_commands(interaction: discord.Interaction): # <-- SEM SELF
    if interaction.user.id != OWNER_ID:
        return await interaction.response.send_message("Apenas o dono.", ephemeral=True)

    cmds = client.tree.get_commands(guild=GUILD_ID_OBJ)
    if not cmds:
        return await interaction.response.send_message("Nenhum comando slash registrado neste servidor.", ephemeral=True)

    lines = [f"`/{c.name}`" + (f" -> `{c.description}`" if c.description else "") for c in cmds]
    text = f"**Comandos Slash ({len(cmds)}):**\n" + "\n".join(lines)

    if len(text)>2000:
        parts=[text[i:i+1990] for i in range(0,len(text),1990)]
        await interaction.response.send_message(parts[0], ephemeral=True)
        for p in parts[1:]:
            await interaction.followup.send(p, ephemeral=True)
    else:
        await interaction.response.send_message(text, ephemeral=True)

@client.tree.command(name="modpanel", description="Abre painel moderação (dono).")
@app_commands.guilds(GUILD_ID_OBJ)
async def modpanel(interaction: discord.Interaction): # <-- SEM SELF
    if interaction.user.id != OWNER_ID:
        return await interaction.response.send_message("❌ Apenas o dono.", ephemeral=True)
    view = ModPanelView()
    embed=discord.Embed(title="Painel Moderação", description="**1.** Selecione user.\n**2.** Escolha ação.", color=discord.Color.blurple())
    await interaction.response.send_message(embed=embed, view=view, ephemeral=True)

@client.tree.command(name="ajuda", description="Mostra o painel de ajuda do Boreal.")
@app_commands.guilds(GUILD_ID_OBJ)
async def ajuda(interaction: discord.Interaction): # <-- SEM SELF
    try:
        embed = discord.Embed(
            title="🌸 Boreal Help",
            description="Oi, tudo bem? me chamo Boreal e estou aqui para te apresentar minhas funcionalidade de um jeitinho bem especial!",
            color=discord.Color.pink() # CORRIGIDO: de .nitro_pink() para .pink()
        )
        embed.add_field(name="Visite meu website:", value="https://boreal.squareweb.app", inline=False)
        embed.add_field(name="Termos de uso:", value="https://boreal.squareweb.app/termos", inline=False)

        if interaction.user.display_avatar:
            embed.set_footer(text=f"Solicitado por {interaction.user.display_name}", icon_url=interaction.user.display_avatar.url)
        else:
             embed.set_footer(text=f"Solicitado por {interaction.user.display_name}")

        view = View()
        view.add_item(Button(label="Adicione-me", url="https://discord.com/oauth2/authorize?client_id=1050964485776605214"))
        view.add_item(Button(label="Meus comandos", url="https://boreal.squareweb.app/comandos"))

        await interaction.response.send_message(embed=embed, view=view, ephemeral=False)
    except Exception as e:
        logger.error(f"Erro no comando /ajuda: {e}", exc_info=True)
        if not interaction.response.is_done():
            await interaction.response.send_message("Ocorreu um erro ao mostrar a ajuda.", ephemeral=True)
        else:
            try:
                await interaction.followup.send("Ocorreu um erro ao mostrar a ajuda.", ephemeral=True)
            except discord.HTTPException:
                pass

# ====================================================================================================
#   LÓGICA DA ECONOMIA (Refatorada para MongoDB)
# ====================================================================================================

class DailyButtonView(discord.ui.View):
    def __init__(self):
        super().__init__(timeout=None) # Persistente

    @discord.ui.button(label="Resgatar Estrelas 🌟",
                        style=discord.ButtonStyle.success,
                        custom_id="persistent_daily_button:v1") # ID Persistente
    async def daily_button_callback(self, interaction: discord.Interaction, button: discord.ui.Button):
        user_id_str = str(interaction.user.id)
        await interaction.response.defer(ephemeral=True)

        try:
            user_profile = await get_profile_bot(user_id_str)
            current_time = time.time()
            COOLDOWN = 86400 # 24 horas

            last_discord_daily = user_profile.get("last_discord_daily", 0.0)

            if (current_time - last_discord_daily) > COOLDOWN:
                amount = random.randint(50, 150) # Recompensa Discord

                # Operação atômica
                result = await client.profiles.find_one_and_update(
                    {"_id": user_id_str},
                    {
                        "$set": {"last_discord_daily": current_time},
                        "$inc": {"estrelas": amount}
                    },
                    return_document=pymongo.ReturnDocument.AFTER
                )

                novo_saldo = result.get("estrelas", 0)
                await interaction.followup.send(f"🎉 Você resgatou **{amount} Estrelas**! Seu novo saldo é **{novo_saldo}** 🌟.", ephemeral=True)

            else:
                remaining_time = COOLDOWN - (current_time - last_discord_daily)
                hours = int(remaining_time // 3600)
                minutes = int((remaining_time % 3600) // 60)
                await interaction.followup.send(f"Você já resgatou seu prêmio. Tente novamente em **{hours}h {minutes}m**.", ephemeral=True)

        except Exception as e:
            logger.error(f"Erro no resgate daily (MongoDB) para {user_id_str}: {e}", exc_info=True)
            await interaction.followup.send("Ocorreu um erro ao processar seu resgate.", ephemeral=True)

@client.tree.command(name="setup_daily_button", description="ADMIN: Posta a mensagem de resgate diário de Estrelas.")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.checks.has_permissions(administrator=True)
async def setup_daily_button(interaction: discord.Interaction): # <-- SEM SELF
    embed = discord.Embed(
        title="🌟 Resgate Diário de Estrelas 🌟",
        description=(
            "Clique no botão abaixo **uma vez por dia** para resgatar suas Estrelas!\n\n"
            "Use `/saldo` para ver quantas você tem e visite nosso site para resgatar um bônus!"
        ),
        color=discord.Color.gold()
    )
    embed.set_thumbnail(url="https://i.imgur.com/vCN5hE5.png")

    try:
        await interaction.channel.send(embed=embed, view=DailyButtonView())
        await interaction.response.send_message("Mensagem de Daily postada!", ephemeral=True)
    except discord.Forbidden:
        await interaction.response.send_message("Não tenho permissão para enviar mensagens ou embeds neste canal.", ephemeral=True)
    except Exception as e:
        logger.error(f"Erro ao postar /setup_daily_button: {e}", exc_info=True)
        await interaction.response.send_message(f"Ocorreu um erro: {e}", ephemeral=True)

@setup_daily_button.error
async def setup_daily_button_error(interaction: discord.Interaction, error: app_commands.AppCommandError):
    if isinstance(error, app_commands.MissingPermissions):
        await interaction.response.send_message("Apenas administradores podem usar este comando.", ephemeral=True)
    else:
        logger.error(f"Erro /setup_daily_button: {error}", exc_info=True)
        if not interaction.response.is_done():
            await interaction.response.send_message("Ocorreu um erro.", ephemeral=True)

@client.tree.command(name="saldo", description="Verifica seu saldo de Estrelas (ou de outro usuário).")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.describe(usuario="O usuário que você quer ver o saldo (opcional).")
async def saldo(interaction: discord.Interaction, usuario: discord.Member = None): # <-- SEM SELF
    target_user = usuario or interaction.user

    try:
        profile = await get_profile_bot(target_user.id)
        estrelas = profile.get("estrelas", 0)

        await interaction.response.send_message(f"O saldo de {target_user.mention} é **{estrelas} Estrelas** 🌟.", ephemeral=True)

    except Exception as e:
        logger.error(f"Erro ao buscar /saldo para {target_user.id}: {e}", exc_info=True)
        await interaction.response.send_message("Não foi possível buscar o saldo.", ephemeral=True)

@client.tree.command(name="pagar", description="Transfere Estrelas para outro usuário.")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.describe(usuario="Para quem você quer pagar.", quantidade="Quantas estrelas (deve ser positivo).")
async def pagar(interaction: discord.Interaction, usuario: discord.Member, quantidade: int): # <-- SEM SELF (CORRIGIDO)
    proposer = interaction.user
    target = usuario

    if proposer.id == target.id:
        return await interaction.response.send_message("Você não pode pagar a si mesmo.", ephemeral=True)
    if target.bot:
         return await interaction.response.send_message("Você não pode pagar um bot.", ephemeral=True)
    if quantidade <= 0:
        return await interaction.response.send_message("A quantidade deve ser positiva.", ephemeral=True)

    try:
        proposer_profile = await get_profile_bot(proposer.id)
        proposer_estrelas = proposer_profile.get("estrelas", 0)

        if proposer_estrelas < quantidade:
            return await interaction.response.send_message(f"Você não tem Estrelas suficientes. Seu saldo: **{proposer_estrelas}** 🌟.", ephemeral=True)

        # Garante que o perfil do alvo exista
        await get_profile_bot(target.id)

        # 1. Remove do proponente
        await client.profiles.update_one(
             {"_id": str(proposer.id)},
             {"$inc": {"estrelas": -quantidade}}
        )
        # 2. Adiciona ao alvo
        await client.profiles.update_one(
            {"_id": str(target.id)},
            {"$inc": {"estrelas": quantidade}}
        )

        await interaction.response.send_message(f"✅ Você transferiu **{quantidade} Estrelas** 🌟 para {target.mention}!", ephemeral=False)

        try:
             await target.send(f"Boas notícias! {proposer.mention} te transferiu **{quantidade} Estrelas** 🌟.")
        except (discord.Forbidden, discord.HTTPException):
             pass # Ignora se a DM estiver fechada

    except Exception as e:
        logger.error(f"Erro no /pagar de {proposer.id} para {target.id} (MongoDB): {e}", exc_info=True)
        await interaction.response.send_message("Ocorreu um erro ao processar o pagamento.", ephemeral=True)

@client.tree.command(name="top_estrelas", description="Mostra o ranking de quem tem mais Estrelas.")
@app_commands.guilds(GUILD_ID_OBJ)
async def top_estrelas(interaction: discord.Interaction): # <-- SEM SELF
    await interaction.response.defer(ephemeral=False)

    try:
        cursor = client.profiles.find({"estrelas": {"$gt": 0}}).sort("estrelas", pymongo.DESCENDING).limit(10)

        embed = discord.Embed(title="🏆 Top 10 - Mais Estrelas 🌟", color=discord.Color.gold())
        description = ""
        rank_emojis = ["🥇", "🥈", "🥉", "4.", "5.", "6.", "7.", "8.", "9.", "10."]

        rank_index = 0
        async for profile in cursor:
            user_id = int(profile["_id"])
            user = interaction.guild.get_member(user_id)
            username = user.mention if user else f"ID: {user_id} (Saiu?)"
            estrelas = profile.get("estrelas", 0)
            description += f"**{rank_emojis[rank_index]}** {username}: **{estrelas}** Estrelas\n"
            rank_index += 1

        if not description:
             description = "Ninguém tem Estrelas ainda. Seja o primeiro!"

        embed.description = description
        await interaction.followup.send(embed=embed)

    except Exception as e:
        logger.error(f"Erro ao gerar /top_estrelas (MongoDB): {e}", exc_info=True)
        await interaction.followup.send("Ocorreu um erro ao buscar o ranking.", ephemeral=True)


@client.tree.command(name="admin_estrelas", description="ADMIN: Modifica as Estrelas de um usuário.")
@app_commands.guilds(GUILD_ID_OBJ)
@app_commands.checks.has_permissions(administrator=True)
@app_commands.describe(acao="Adicionar, Remover ou Definir o valor.", usuario="O usuário a ser modificado.", quantidade="O valor (deve ser positivo).")
@app_commands.choices(acao=[
    app_commands.Choice(name="Adicionar", value="add"),
    app_commands.Choice(name="Remover", value="remove"),
    app_commands.Choice(name="Definir", value="set")
])
async def admin_estrelas(interaction: discord.Interaction, acao: app_commands.Choice[str], usuario: discord.Member, quantidade: int): # <-- SEM SELF
    if quantidade < 0:
        return await interaction.response.send_message("A quantidade não pode ser negativa.", ephemeral=True)

    user_id_str = str(usuario.id)
    acao_value = acao.value

    try:
        # Garante que o perfil existe
        profile = await get_profile_bot(user_id_str)

        operation = None
        if acao_value == "add":
            operation = {"$inc": {"estrelas": quantidade}}
        elif acao_value == "remove":
            current_stars = profile.get("estrelas", 0)
            final_amount = max(0, current_stars - quantidade) # Garante que o mínimo é 0
            operation = {"$set": {"estrelas": final_amount}}
        elif acao_value == "set":
            operation = {"$set": {"estrelas": quantidade}}

        if not operation:
             return await interaction.response.send_message("Ação inválida.", ephemeral=True)

        result = await client.profiles.find_one_and_update(
            {"_id": user_id_str},
            operation,
            return_document=pymongo.ReturnDocument.AFTER
        )

        novo_saldo = result.get("estrelas", 0)
        await interaction.response.send_message(
            f"✅ Ação `{acao.name}` aplicada. O novo saldo de {usuario.mention} é **{novo_saldo} Estrelas** 🌟.",
            ephemeral=True
        )

    except Exception as e:
         logger.error(f"Erro no /admin_estrelas (MongoDB): {e}", exc_info=True)
         await interaction.response.send_message("Ocorreu um erro ao modificar o saldo.", ephemeral=True)

@admin_estrelas.error
async def admin_estrelas_error(interaction: discord.Interaction, error: app_commands.AppCommandError):
    if isinstance(error, app_commands.MissingPermissions):
        await interaction.response.send_message("Apenas administradores podem usar este comando.", ephemeral=True)
    else:
        logger.error(f"Erro /admin_estrelas: {error}", exc_info=True)
        if not interaction.response.is_done():
            await interaction.response.send_message("Ocorreu um erro.", ephemeral=True)

# ====================================================================================================
#   INICIAR O BOT
# ====================================================================================================
if __name__ == "__main__":
    # Verificações de arquivos essenciais ANTES de rodar
    # (Caminhos relativos assumem que o bot é iniciado da pasta /application/)
    essential_files = ["spotify_background.png", "arial.ttf", "arialbd.ttf", os.path.join(IMAGE_FOLDER, "designer.png")]
    missing = [f for f in essential_files if not os.path.exists(f)]
    if missing:
        logger.error(f"Caminho atual: {os.getcwd()}")
        raise FileNotFoundError(f"Arquivos essenciais faltando: {', '.join(missing)}. O bot não pode iniciar.")

    if not os.path.isdir(IMAGE_FOLDER):
         raise FileNotFoundError(f"A pasta '{IMAGE_FOLDER}' é necessária mas não foi encontrada.")

    if not any(os.path.exists(os.path.join(IMAGE_FOLDER, "bandeiras", f"{loc}.png")) for loc in LOCALIZACOES_VALIDAS):
         logger.warning(f"Nenhuma imagem de bandeira encontrada em '{IMAGE_FOLDER}/bandeiras/'. O comando /perfil não mostrará bandeiras.")


    if str(OWNER_ID) == "1017254600480411670":
        logger.warning("OWNER_ID padrão detectado (POZEz). Certifique-se que isso está correto.")

    logger.info('Iniciando o bot...')
    client.run(DISCORD_TOKEN)